## Install Data Augmentation Libraries

### Subtask:
Install necessary Python libraries, such as `imgaug`, which will be used for performing various image transformations and augmenting COCO annotations (segmentation masks and bounding boxes) to create synthetic data. This ensures all required tools are available for the data generation process.


**Reasoning**:
The user wants to install necessary Python libraries for data augmentation. I will use `pip install` within a code block to install `imgaug`, `opencv-python`, and `scikit-image`.



In [1]:
pip install imgaug opencv-python scikit-image

**Reasoning**:
The subtask requires modifying the print statements within the `if imgs_count > 0:` block of cell `TPUKkd36GGgz` to improve phrasing and clarity. I will provide the full updated code for that cell, specifically focusing on the final success message.



In [2]:
from google.colab import drive
import os, json, shutil, zipfile
from pathlib import Path

# ================= MONTAR GOOGLE DRIVE =================
drive.mount('/content/drive')

# ================= CONFIGURACIÓN =================
# Ruta al ZIP en Google Drive
zip_path = '/content/drive/MyDrive/alopecia_project/data/test-coco-upload.zip'  # ¡AJUSTA ESTA RUTA!

# Ruta donde quieres guardar el dataset YOLO en Drive
output_dir = '/content/drive/MyDrive/alopecia_project/yolo_dataset'

# ================= EXTRAER ZIP =================
print("📦 Extrayendo ZIP de Google Drive...")
extract_dir = '/content/temp_extract'
shutil.rmtree(extract_dir, ignore_errors=True)
os.makedirs(extract_dir)

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Listar contenido
        print("Contenido del ZIP:")
        file_list = zip_ref.namelist()
        for file in file_list[:10]:  # Mostrar primeros 10
            print(f"  - {file}")

        if len(file_list) > 10:
            print(f"  ... y {len(file_list) - 10} más")

        # Extraer TODO
        zip_ref.extractall(extract_dir)

    print(f"✅ ZIP extraído en: {extract_dir}")

except FileNotFoundError:
    print(f"❌ No se encontró el ZIP en: {zip_path}")
    print("\n📁 Buscando archivos ZIP en Google Drive...")

    # Buscar archivos ZIP
    import glob
    zip_files = glob.glob('/content/drive/MyDrive/**/*.zip', recursive=True)
    if zip_files:
        print("Archivos ZIP encontrados:")
        for zf in zip_files[:5]:
            print(f"  - {zf}")
        zip_path = zip_files[0]
        print(f"\n✅ Usando: {zip_path}")

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
    else:
        raise FileNotFoundError("No se encontró ningún archivo ZIP")

# ================= BUSCAR ARCHIVOS =================
# Buscar JSON
json_files = list(Path(extract_dir).rglob('*.json'))
if not json_files:
    raise FileNotFoundError("❌ No se encontró JSON en el ZIP")

json_path = str(json_files[0])
print(f"✅ JSON encontrado: {json_path}")

# Buscar carpeta de imágenes
images_dir = None
possible_dirs = ['images', 'img', 'upload', 'data', 'media']
for dir_name in possible_dirs:
    dir_path = Path(extract_dir) / dir_name
    if dir_path.exists() and any(dir_path.iterdir()):
        images_dir = dir_path
        print(f"✅ Carpeta de imágenes encontrada: {images_dir}")
        break

# Si no, buscar cualquier carpeta con imágenes
if not images_dir:
    for root, dirs, files in os.walk(extract_dir):
        if any(f.lower().endswith(('.jpg', '.png', '.jpeg')) for f in files):
            images_dir = Path(root)
            print(f"✅ Imágenes encontradas en: {images_dir}")
            break

if not images_dir:
    print("⚠️ No se encontró carpeta de imágenes específica, buscando en todo el extract...")
    images_dir = Path(extract_dir)

# Listar imágenes encontradas
imagenes = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.PNG']:
    imagenes.extend(list(images_dir.rglob(ext)))

print(f"📸 Total imágenes encontradas: {len(imagenes)}")
for img in imagenes[:5]:
    print(f"  - {img.name}")

# ================= CARGAR JSON =================
with open(json_path, 'r') as f:
    coco_data = json.load(f) # Renamed 'datos' to 'coco_data' to reflect COCO format

print(f"\n📊 Total de imágenes en JSON (COCO): {len(coco_data.get('images', []))}")
print(f"📊 Total de anotaciones en JSON (COCO): {len(coco_data.get('annotations', []))}")

# Mostrar estructura del JSON para depuración
if coco_data and isinstance(coco_data, dict):
    print("\n📝 Estructura del JSON (claves principales):")
    for key in coco_data.keys():
        print(f"  - {key}")
    if 'images' in coco_data and coco_data['images']:
        print("\n📝 Estructura del primer elemento de 'images':")
        for key in list(coco_data['images'][0].keys())[:10]:
            print(f"  - {key}")
    if 'annotations' in coco_data and coco_data['annotations']:
        print("\n📝 Estructura del primer elemento de 'annotations':")
        for key in list(coco_data['annotations'][0].keys())[:10]:
            print(f"  - {key}")

# ================= CREAR ESTRUCTURA YOLO =================
yolo_dir = Path(output_dir)
shutil.rmtree(yolo_dir, ignore_errors=True)  # Limpiar si existe
(yolo_dir / 'images').mkdir(parents=True, exist_ok=True)
(yolo_dir / 'labels').mkdir(parents=True, exist_ok=True)

# Mapear categorías a IDs de clase y almacenar nombres de clase
class_names = [cat['name'] for cat in coco_data.get('categories', [])]
class_id_map = {cat['id']: i for i, cat in enumerate(coco_data.get('categories', []))}

# Mapear image_id a información de imagen para búsqueda rápida
image_info_map = {img['id']: img for img in coco_data.get('images', [])}

procesadas = 0
imagenes_usadas = set()

# Iterar sobre cada imagen en el JSON COCO
for i, img_data in enumerate(coco_data.get('images', [])):
    image_id = img_data['id']
    nombre_imagen_json = img_data.get('file_name')
    img_width = img_data.get('width')
    img_height = img_data.get('height')

    if not nombre_imagen_json or img_width is None or img_height is None:
        print(f"❌ Información incompleta para imagen ID {image_id}. Saltando.")
        continue

    print(f"\n--- Procesando imagen {i+1}/{len(coco_data['images'])} (ID: {image_id}) ---")

    # Limpiar nombre (quitar rutas, parámetros URL, etc.)
    nombre_limpio = os.path.basename(str(nombre_imagen_json))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    print(f"🔍 Buscando imagen: '{nombre_limpio}'")

    imagen_encontrada = None
    # Prioridad: Coincidencia exacta con nombre de archivo
    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            imagen_encontrada = img_path_candidate
            break
    # Si no se encuentra, intentar por nombre base sin extensión
    if not imagen_encontrada:
        nombre_base = os.path.splitext(nombre_limpio)[0]
        for img_path_candidate in imagenes:
            if os.path.splitext(img_path_candidate.name)[0] == nombre_base:
                imagen_encontrada = img_path_candidate
                break
    # Si aún no se encuentra, intentar coincidencia parcial con nombres limpios
    if not imagen_encontrada:
        nombre_limpio_lower = nombre_limpio.lower()
        for img_path_candidate in imagenes:
            img_name_lower = img_path_candidate.name.lower()
            if nombre_limpio_lower in img_name_lower or img_name_lower in img_path_candidate.name.lower():
                imagen_encontrada = img_path_candidate
                break
            if '-' in nombre_limpio_lower: # Handle hash prefixes (e.g., "c6483cef-")
                nombre_sin_hash = nombre_limpio_lower.split('-', 1)[1]
                if nombre_sin_hash in img_name_lower:
                    imagen_encontrada = img_path_candidate
                    break

    if imagen_encontrada: # Proceed only if image file is found
        if imagen_encontrada in imagenes_usadas:
            print(f"⚠️  Imagen ya procesada (posible duplicado en JSON o nombres): {imagen_encontrada.name}")
            continue # Skip to avoid creating duplicate YOLO entries

        print(f"✅ Imagen encontrada: {imagen_encontrada.name}")
        imagenes_usadas.add(imagen_encontrada)

        # 3. Copiar imagen
        ext = imagen_encontrada.suffix
        img_dest = yolo_dir / 'images' / f"{image_id}{ext}" # Use COCO image ID as filename
        shutil.copy(imagen_encontrada, img_dest)

        # 4. Crear etiquetas YOLO
        txt_path = yolo_dir / 'labels' / f"{image_id}.txt"

        with open(txt_path, 'w') as f_txt:
            anotaciones_escritas = 0
            # Filtrar anotaciones para esta imagen
            image_annotations = [ann for ann in coco_data.get('annotations', []) if ann['image_id'] == image_id]

            for ann in image_annotations:
                if 'bbox' in ann: # Process bounding box annotations
                    x_min, y_min, ann_width, ann_height = ann['bbox']

                    # Convert to normalized YOLO format
                    center_x = (x_min + ann_width / 2) / img_width
                    center_y = (y_min + ann_height / 2) / img_height
                    norm_width = ann_width / img_width
                    norm_height = ann_height / img_height

                    category_id = ann.get('category_id')
                    if category_id is None or category_id not in class_id_map:
                        print(f"   ❌ Categoría ID {category_id} no encontrada en 'categories'. Saltando anotación.")
                        continue

                    class_idx = class_id_map[category_id] # YOLO class index

                    f_txt.write(f"{class_idx} {center_x:.6f} {center_y:.6f} {norm_width:.6f} {norm_height:.6f}\n")
                    anotaciones_escritas += 1
                    class_name_display = class_names[class_idx] if class_idx < len(class_names) else f"ID {category_id}"
                    print(f"   🎯 Anotación BBOX: {class_name_display} - ({center_x:.2f}, {center_y:.2f}, {norm_width:.2f}, {norm_height:.2f})")
                elif 'segmentation' in ann and ann['segmentation']:
                    print(f"   ⚠️ Se encontró anotación de segmentación para ID {image_id}, pero el script actual solo procesa 'bbox' a YOLO.")

            if anotaciones_escritas == 0:
                print(f"⚠️  No se encontraron anotaciones BBOX válidas para esta imagen")
                # Create an empty label file if no valid annotations were processed
                with open(txt_path, 'w') as f_txt:
                    f_txt.write("")

        procesadas += 1
    else:
        print(f"❌ Imagen NO encontrada en el directorio de extracción para '{nombre_limpio}'")

# ================= CREAR dataset.yaml =================
with open(yolo_dir / 'dataset.yaml', 'w') as f:
    f.write(f"# Dataset Alopecia - YOLO\n")
    f.write(f"path: {output_dir}\n")
    f.write(f"train: images\n")
    f.write(f"val: images\n\n")
    f.write(f"# Clases\n")
    f.write(f"nc: {len(class_names)}\n")
    f.write(f"names: {class_names}\n")

# ================= RESUMEN =================
print(f"\n{'='*50}")
print(f"🎉 PROCESO COMPLETADO")
print(f"{'='*50}")
print(f"✅ Imágenes en JSON (COCO): {len(coco_data.get('images', []))}")
print(f"✅ Anotaciones en JSON (COCO): {len(coco_data.get('annotations', []))}")
print(f"✅ Imágenes procesadas y copiadas: {procesadas}")
print(f"✅ Clases encontradas: {class_names}")
print(f"✅ Total clases: {len(class_names)}")

# Estadísticas
imgs_count = len(list((yolo_dir / 'images').glob('*')))
labels_count = len(list((yolo_dir / 'labels').glob('*.txt')))

print(f"\n📊 Estadísticas finales:")
print(f"   Imágenes copiadas a YOLO: {imgs_count}")
print(f"   Etiquetas creadas en YOLO: {labels_count}")

# Mostrar ubicación final
print(f"\n📁 Dataset YOLO creado en Google Drive:")
print(f"   {output_dir}/")
print(f"   ├── images/")
print(f"   ├── labels/")
print(f"   └── dataset.yaml")

# Verificar acceso
if imgs_count > 0:
    print(f"\n✅ El dataset YOLO ha sido preparado con éxito y está listo para el entrenamiento.")
    print(f"   Asegúrate de que los archivos se encuentren en las siguientes ubicaciones:")
    print(f"   - Ruta del dataset: {output_dir}")
    print(f"   - Imágenes de entrenamiento: {imgs_count} archivos en {output_dir}/images/")
    print(f"   - Archivos de etiquetas (bounding boxes): {labels_count} archivos en {output_dir}/labels/")
else:
    print(f"\n⚠️  No se procesaron imágenes para el dataset YOLO. Por favor, verifica:")
    print(f"   - Que el ZIP contenga imágenes y anotaciones COCO válidas con 'bbox'.")
    print(f"   - Que los nombres de las imágenes en el COCO JSON coincidan con los archivos de imagen extraídos.")

# Limpiar temporal
# shutil.rmtree(extract_dir, ignore_errors=True) # Commented out to prevent premature deletion
print(f"\n🧹 Directorio temporal conservado para procesamiento posterior.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📦 Extrayendo ZIP de Google Drive...
Contenido del ZIP:
  - test-coco/
  - test-coco/images/
  - test-coco/images/133e41f0-imgi_460_alopecia.jpg
  - test-coco/images/26aee95b-imgi_454_areata5.jpg
  - test-coco/images/3022b173-imgi_400_Hair-loss_SP-Feature.jpg
  - test-coco/images/769446b8-imgi_413_alopecia-androgenica-femenina-mujer-joven.jpg
  - test-coco/images/79b0bbb7-imgi_429_alopecia-areata.jpg.webp
  - test-coco/images/7a30afe5-imgi_188_persona-con-alopecia.jpg
  - test-coco/images/7de3a7df-imgi_109_images.jpg
  - test-coco/images/7e88f071-imgi_404_Alopecia-en-adolescentes.jpg
  ... y 8 más
✅ ZIP extraído en: /content/temp_extract
✅ JSON encontrado: /content/temp_extract/test-coco/result_coco.json
✅ Imágenes encontradas en: /content/temp_extract/test-coco/images
📸 Total imágenes encontradas: 15
  - 7a30afe5-imgi_188_persona-con-alopecia.jpg
  - 7de3a7df

### Confirmation of Updated Message

The updated output message in the code cell `TPUKkd36GGgz` is now clear and accurately reflects the dataset's readiness for YOLO training.

The improvements include:
1.  **Clearer affirmative statement**: "✅ El dataset YOLO ha sido preparado con éxito y está listo para el entrenamiento." (The YOLO dataset has been successfully prepared and is ready for training.)
2.  **Explicit instruction**: "Asegúrate de que los archivos se encuentren en las siguientes ubicaciones:" (Ensure that the files are in the following locations:)
3.  **Specific details**: The message now clearly itemizes the path to the dataset, the exact count and location of the training images, and the exact count and location of the bounding box label files. This provides comprehensive information to the user, ensuring they know precisely where to find their prepared dataset components for the next training steps.

This level of detail is crucial for usability and reduces ambiguity when proceeding with YOLO model training.

## Generate Augmented Dataset

### Subtask:
Generate approximately 100 synthetic images and their corresponding COCO annotations by applying various geometric and photometric transformations to the initial 10 images. Ensure COCO annotations (segmentation polygons) are transformed consistently with the images. Save the augmented data in a new directory structure, similar to the `aligned_dataset`, for the segmentation model's training.


**Reasoning**:
The first instruction is to define the input and output directories and initialize the `imgaug` augmentation pipeline. I also need to load the `coco_data` and `imagenes` from the previous steps to begin the augmentation process. The augmentation sequence will include geometric and photometric transformations.



In [1]:
print('🔄 Downgrading numpy for imgaug compatibility...')
!pip uninstall -y numpy
!pip install numpy==1.26.4
print('✅ numpy downgraded to 1.26.4.')


🔄 Downgrading numpy for imgaug compatibility...
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-

✅ numpy downgraded to 1.26.4.


In [1]:
import os
import json
import shutil
from pathlib import Path
import cv2
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

# ================= 1. DEFINIR DIRECTORIOS Y CONFIGURACIÓN =================
# Input: Directorio de extracción temporal del ZIP y JSON COCO original
# `extract_dir` y `json_path` deben venir de la celda de extracción (TPUKkd36GGgz)
# `coco_data` y `imagenes` también de la celda TPUKkd36GGgz
# `class_names` de la celda TPUKkd36GGgz

# Output: Directorio para el dataset aumentado
augmented_output_dir = Path('/content/alopecia_dataset/augmented_dataset')
augmented_img_dir = augmented_output_dir / 'images'
augmented_annot_dir = augmented_output_dir / 'annotations'

# Limpiar y crear directorios
shutil.rmtree(augmented_output_dir, ignore_errors=True)
augmented_img_dir.mkdir(parents=True, exist_ok=True)
augmented_annot_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Directorios para el dataset aumentado creados en: {augmented_output_dir}")

# ================= 2. INICIALIZAR AUMENTADORES imgaug =================
# Definir la secuencia de aumentación. Usamos `random_order=True` para variar el orden.
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # Volteo horizontal con 50% de probabilidad
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # Escala 80-120%
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # Traslación -20% a +20%
        rotate=(-25, 25), # Rotación -25 a +25 grados
        shear=(-8, 8) # Cizallamiento -8 a +8 grados
    ),
    iaa.Multiply((0.8, 1.2), per_channel=0.2), # Ajuste de brillo (80-120%)
    iaa.LinearContrast((0.75, 1.5)), # Ajuste de contraste
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # Ruido Gaussiano
    iaa.Sometimes(0.2, iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 1.5))), # Afilado
    iaa.Sometimes(0.2, iaa.Emboss(alpha=(0.0, 1.0), strength=(0.5, 1.5))), # Relieve
    iaa.Sometimes(0.2, iaa.EdgeDetect(alpha=(0.0, 0.7))), # Detección de bordes
    iaa.Sometimes(0.2, iaa.MotionBlur(k=(3, 7), angle=[-45, 45])), # Desenfoque de movimiento
    iaa.Sometimes(0.2, iaa.AddToHueAndSaturation((-20, 20))) # Cambios de tono y saturación
], random_order=True) # Aplicar aumentaciones en orden aleatorio

print("✅ Secuencia de aumentación de imgaug inicializada.")

# ================= 3. & 4. CARGAR COCO ORIGINAL Y MAPPING =================
# `coco_data` y `imagenes` ya están disponibles desde la ejecución anterior.
# Asegurémonos de tener un mapa de image_id a info de imagen y anotaciones

original_images_info = {img['id']: img for img in coco_data.get('images', [])}
original_annotations_info = {}
for ann in coco_data.get('annotations', []):
    image_id = ann['image_id']
    if image_id not in original_annotations_info:
        original_annotations_info[image_id] = []
    original_annotations_info[image_id].append(ann)

print(f"✅ COCO original cargado. Imágenes: {len(original_images_info)}, Anotaciones para {len(original_annotations_info)} imágenes.")

✅ Directorios para el dataset aumentado creados en: /content/alopecia_dataset/augmented_dataset
✅ Secuencia de aumentación de imgaug inicializada.


NameError: name 'coco_data' is not defined

**Reasoning**:
The previous code failed because `imgaug` is not compatible with `numpy` version 2.0 or newer, as indicated by the `AttributeError: np.sctypes` was removed. I need to downgrade `numpy` to a compatible version (e.g., `1.26.4`) before `imgaug` can be used. This will be done in a new cell, followed by the original code block.



**Reasoning**:
Numpy has been downgraded, so now I will re-run the code to initialize `imgaug` and set up the directories and mappings for the augmented dataset. This should now execute without the `AttributeError`.



In [3]:
import os
import json
import shutil
from pathlib import Path
import cv2
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

# ================= 1. DEFINIR DIRECTORIOS Y CONFIGURACIÓN =================
# Input: Directorio de extracción temporal del ZIP y JSON COCO original
# `extract_dir` y `json_path` deben venir de la celda de extracción (TPUKkd36GGgz)
# `coco_data` y `imagenes` también de la celda TPUKkd36GGgz
# `class_names` de la celda TPUKkd36GGgz

# Output: Directorio para el dataset aumentado
augmented_output_dir = Path('/content/alopecia_dataset/augmented_dataset')
augmented_img_dir = augmented_output_dir / 'images'
augmented_annot_dir = augmented_output_dir / 'annotations'

# Limpiar y crear directorios
shutil.rmtree(augmented_output_dir, ignore_errors=True)
augmented_img_dir.mkdir(parents=True, exist_ok=True)
augmented_annot_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Directorios para el dataset aumentado creados en: {augmented_output_dir}")

# ================= 2. INICIALIZAR AUMENTADORES imgaug =================
# Definir la secuencia de aumentación. Usamos `random_order=True` para variar el orden.
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # Volteo horizontal con 50% de probabilidad
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # Escala 80-120%
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # Traslación -20% a +20%
        rotate=(-25, 25), # Rotación -25 a +25 grados
        shear=(-8, 8) # Cizallamiento -8 a +8 grados
    ),
    iaa.Multiply((0.8, 1.2), per_channel=0.2), # Ajuste de brillo (80-120%)
    iaa.LinearContrast((0.75, 1.5)), # Ajuste de contraste
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # Ruido Gaussiano
    iaa.Sometimes(0.2, iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 1.5))), # Afilado
    iaa.Sometimes(0.2, iaa.Emboss(alpha=(0.0, 1.0), strength=(0.5, 1.5))), # Relieve
    iaa.Sometimes(0.2, iaa.EdgeDetect(alpha=(0.0, 0.7))), # Detección de bordes
    iaa.Sometimes(0.2, iaa.MotionBlur(k=(3, 7), angle=[-45, 45])), # Desenfoque de movimiento
    iaa.Sometimes(0.2, iaa.AddToHueAndSaturation((-20, 20))) # Cambios de tono y saturación
], random_order=True) # Aplicar aumentaciones en orden aleatorio

print("✅ Secuencia de aumentación de imgaug inicializada.")

# ================= 3. & 4. CARGAR COCO ORIGINAL Y MAPPING =================
# `coco_data` y `imagenes` ya están disponibles desde la ejecución anterior.
# Asegurémonos de tener un mapa de image_id a info de imagen y anotaciones

original_images_info = {img['id']: img for img in coco_data.get('images', [])}
original_annotations_info = {}
for ann in coco_data.get('annotations', []):
    image_id = ann['image_id']
    if image_id not in original_annotations_info:
        original_annotations_info[image_id] = []
    original_annotations_info[image_id].append(ann)

print(f"✅ COCO original cargado. Imágenes: {len(original_images_info)}, Anotaciones para {len(original_annotations_info)} imágenes.")


✅ Directorios para el dataset aumentado creados en: /content/alopecia_dataset/augmented_dataset
✅ Secuencia de aumentación de imgaug inicializada.


NameError: name 'coco_data' is not defined

**Reasoning**:
The previous cell failed because `coco_data` and `imagenes` were not defined after the numpy downgrade. I need to re-load these variables from their respective paths (`json_path` and `images_dir`) to make them accessible for the data augmentation process. I'll add the necessary loading steps at the beginning of the cell.



In [4]:
import os
import json
import shutil
from pathlib import Path
import cv2
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

# ================= RELOAD NECESSARY DATA FROM PREVIOUS CELLS =================
# These variables were defined in cell TPUKkd36GGgz and might have been lost
# after runtime restart or numpy downgrade affecting the environment.

# Re-define paths if not globally available, assuming previous cell execution state
# (e.g., zip_path, output_dir, extract_dir, json_path, images_dir are defined from TPUKkd36GGgz)
# If running this cell independently, ensure these paths are correctly set.

# For demonstration, explicitly re-loading what's needed based on original notebook flow:
# Assumed: `json_path` and `images_dir` from TPUKkd36GGgz are still valid string paths.

# Load `coco_data`
with open(json_path, 'r') as f:
    coco_data = json.load(f)
print(f"✅ `coco_data` re-loaded from {json_path}.")

# Re-list `imagenes` (image paths found during extraction)
imagenes = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.PNG']:
    imagenes.extend(list(Path(images_dir).rglob(ext)))
print(f"✅ `imagenes` re-listed: {len(imagenes)} found in {images_dir}.")

# ================= 1. DEFINIR DIRECTORIOS Y CONFIGURACIÓN =================
# Input: Directorio de extracción temporal del ZIP y JSON COCO original
# `extract_dir` y `json_path` deben venir de la celda de extracción (TPUKkd36GGgz)
# `coco_data` y `imagenes` también de la celda TPUKkd36GGgz
# `class_names` de la celda TPUKkd36GGgz

# Output: Directorio para el dataset aumentado
augmented_output_dir = Path('/content/alopecia_dataset/augmented_dataset')
augmented_img_dir = augmented_output_dir / 'images'
augmented_annot_dir = augmented_output_dir / 'annotations'

# Limpiar y crear directorios
shutil.rmtree(augmented_output_dir, ignore_errors=True)
augmented_img_dir.mkdir(parents=True, exist_ok=True)
augmented_annot_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Directorios para el dataset aumentado creados en: {augmented_output_dir}")

# ================= 2. INICIALIZAR AUMENTADORES imgaug =================
# Definir la secuencia de aumentación. Usamos `random_order=True` para variar el orden.
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # Volteo horizontal con 50% de probabilidad
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # Escala 80-120%
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # Traslación -20% a +20%
        rotate=(-25, 25), # Rotación -25 a +25 grados
        shear=(-8, 8) # Cizallamiento -8 a +8 grados
    ),
    iaa.Multiply((0.8, 1.2), per_channel=0.2), # Ajuste de brillo (80-120%)
    iaa.LinearContrast((0.75, 1.5)), # Ajuste de contraste
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # Ruido Gaussiano
    iaa.Sometimes(0.2, iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 1.5))), # Afilado
    iaa.Sometimes(0.2, iaa.Emboss(alpha=(0.0, 1.0), strength=(0.5, 1.5))), # Relieve
    iaa.Sometimes(0.2, iaa.EdgeDetect(alpha=(0.0, 0.7))), # Detección de bordes
    iaa.Sometimes(0.2, iaa.MotionBlur(k=(3, 7), angle=[-45, 45])), # Desenfoque de movimiento
    iaa.Sometimes(0.2, iaa.AddToHueAndSaturation((-20, 20))) # Cambios de tono y saturación
], random_order=True) # Aplicar aumentaciones en orden aleatorio

print("✅ Secuencia de aumentación de imgaug inicializada.")

# ================= 3. & 4. CARGAR COCO ORIGINAL Y MAPPING =================
# `coco_data` y `imagenes` ya están disponibles desde la ejecución anterior.
# Asegurémonos de tener un mapa de image_id a info de imagen y anotaciones

original_images_info = {img['id']: img for img in coco_data.get('images', [])}
original_annotations_info = {}
for ann in coco_data.get('annotations', []):
    image_id = ann['image_id']
    if image_id not in original_annotations_info:
        original_annotations_info[image_id] = []
    original_annotations_info[image_id].append(ann)

print(f"✅ COCO original cargado. Imágenes: {len(original_images_info)}, Anotaciones para {len(original_annotations_info)} imágenes.")

NameError: name 'json_path' is not defined

**Reasoning**:
The previous execution failed because `json_path` and `images_dir` were undefined. I need to explicitly define these variables with their expected values, which can be inferred from the output of the initial data extraction cell (`TPUKkd36GGgz`). This will ensure `coco_data` and `imagenes` can be re-loaded successfully.



In [5]:
import os
import json
import shutil
from pathlib import Path
import cv2
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

# ================= RELOAD NECESSARY DATA FROM PREVIOUS CELLS =================
# These variables were defined in cell TPUKkd36GGgz and might have been lost
# after runtime restart or numpy downgrade affecting the environment.

# Explicitly define `json_path` and `images_dir` based on prior execution's output
json_path = '/content/temp_extract/test-coco/result_coco.json'
images_dir = '/content/temp_extract/test-coco/images'
extract_dir = '/content/temp_extract'

# Load `coco_data`
with open(json_path, 'r') as f:
    coco_data = json.load(f)
print(f"✅ `coco_data` re-loaded from {json_path}.")

# Re-list `imagenes` (image paths found during extraction)
imagenes = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.PNG']:
    imagenes.extend(list(Path(images_dir).rglob(ext)))
print(f"✅ `imagenes` re-listed: {len(imagenes)} found in {images_dir}.")

# ================= 1. DEFINIR DIRECTORIOS Y CONFIGURACIÓN =================
# Output: Directorio para el dataset aumentado
augmented_output_dir = Path('/content/alopecia_dataset/augmented_dataset')
augmented_img_dir = augmented_output_dir / 'images'
augmented_annot_dir = augmented_output_dir / 'annotations'

# Limpiar y crear directorios
shutil.rmtree(augmented_output_dir, ignore_errors=True)
augmented_img_dir.mkdir(parents=True, exist_ok=True)
augmented_annot_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Directorios para el dataset aumentado creados en: {augmented_output_dir}")

# ================= 2. INICIALIZAR AUMENTADORES imgaug =================
# Definir la secuencia de aumentación. Usamos `random_order=True` para variar el orden.
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # Volteo horizontal con 50% de probabilidad
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # Escala 80-120%
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # Traslación -20% a +20%
        rotate=(-25, 25), # Rotación -25 a +25 grados
        shear=(-8, 8) # Cizallamiento -8 a +8 grados
    ),
    iaa.Multiply((0.8, 1.2), per_channel=0.2), # Ajuste de brillo (80-120%)
    iaa.LinearContrast((0.75, 1.5)), # Ajuste de contraste
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # Ruido Gaussiano
    iaa.Sometimes(0.2, iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 1.5))), # Afilado
    iaa.Sometimes(0.2, iaa.Emboss(alpha=(0.0, 1.0), strength=(0.5, 1.5))), # Relieve
    iaa.Sometimes(0.2, iaa.EdgeDetect(alpha=(0.0, 0.7))), # Detección de bordes
    iaa.Sometimes(0.2, iaa.MotionBlur(k=(3, 7), angle=[-45, 45])), # Desenfoque de movimiento
    iaa.Sometimes(0.2, iaa.AddToHueAndSaturation((-20, 20))) # Cambios de tono y saturación
], random_order=True) # Aplicar aumentaciones en orden aleatorio

print("✅ Secuencia de aumentación de imgaug inicializada.")

# ================= 3. & 4. CARGAR COCO ORIGINAL Y MAPPING =================
# `coco_data` y `imagenes` ya están disponibles desde la ejecución anterior.
# Asegurémonos de tener un mapa de image_id a info de imagen y anotaciones

original_images_info = {img['id']: img for img in coco_data.get('images', [])}
original_annotations_info = {}
for ann in coco_data.get('annotations', []):
    image_id = ann['image_id']
    if image_id not in original_annotations_info:
        original_annotations_info[image_id] = []
    original_annotations_info[image_id].append(ann)

print(f"✅ COCO original cargado. Imágenes: {len(original_images_info)}, Anotaciones para {len(original_annotations_info)} imágenes.")

✅ `coco_data` re-loaded from /content/temp_extract/test-coco/result_coco.json.
✅ `imagenes` re-listed: 15 found in /content/temp_extract/test-coco/images.
✅ Directorios para el dataset aumentado creados en: /content/alopecia_dataset/augmented_dataset
✅ Secuencia de aumentación de imgaug inicializada.
✅ COCO original cargado. Imágenes: 12, Anotaciones para 12 imágenes.


**Reasoning**:
The next step is to iterate through each original image, convert its COCO segmentation annotations into `imgaug` compatible format (SegmentationMapsOnImage), and prepare for augmentation. This involves finding the corresponding image file and processing its segmentation data.



In [6]:
ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
new_image_id = max([img['id'] for img in coco_data['images']]) + 1 if coco_data.get('images') else 0
new_annotation_id = max([ann['id'] for ann in coco_data['annotations']]) + 1 if coco_data.get('annotations') else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            # imgaug expects polygons as a list of points [[x1, y1], [x2, y2], ...]
            # COCO segmentation for polygons is [x1, y1, x2, y2, ...]
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    # Reshape [x1, y1, x2, y2, ...] to [[x1, y1], [x2, y2], ...]
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points=points, label=ann['category_id']))

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas. Se copiará tal cual.")
        # Add original image and its annotations to the augmented dataset
        augmented_coco_data["images"].append(original_img_info)
        for ann in original_annotations:
            augmented_coco_data["annotations"].append(ann)

        augmented_dataset_info_list.append({
            'id': original_img_info['id'],
            'aligned_image': original_img_info['file_name'],
            'annotation_count': len(original_annotations)
        })
        continue

    print(f"  Procesando original ID {original_img_id} ('{original_file_name}') con {len(segmentations_ia)} segmentaciones.")

    # Add original image and its annotations to the augmented dataset (as the first 'augmented' version, X=0)
    augmented_coco_data["images"].append(original_img_info)
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_info['id'],
        'aligned_image': original_img_info['file_name'],
        'annotation_count': len(original_annotations)
    })

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        # Make a copy of the sequence to ensure different random operations for each augmentation
        seq_det = seq.to_deterministic()

        # Augment image and polygons
        image_aug = seq_det.augment_image(image_original)
        polygons_aug = seq_det.augment_polygons(segmentations_ia)

        # Generate new filenames and IDs
        new_image_filename = f"{Path(original_file_name).stem}_aug{aug_idx}{Path(original_file_name).suffix}"
        current_new_image_id = new_image_id
        new_image_id += 1

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.points) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = poly_ia.points.flatten().tolist()
                x_min, y_min, x_max, y_max = poly_ia.bounding_box.x1,
                                             poly_ia.bounding_box.y1,
                                             poly_ia.bounding_box.x2,
                                             poly_ia.bounding_box.y2
                bbox_aug = [x_min, y_min, x_max - x_min, y_max - y_min]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": poly_ia.area,
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")

IndentationError: unexpected indent (ipython-input-1662541983.py, line 125)

**Reasoning**:
The previous code failed due to an `IndentationError` around line 125. This means there's an incorrect spacing in the code. I will fix the indentation to ensure the `bbox_aug` assignment is properly formatted.



In [ ]:
ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
new_image_id = max([img['id'] for img in coco_data['images']]) + 1 if coco_data.get('images') else 0
new_annotation_id = max([ann['id'] for ann in coco_data['annotations']]) + 1 if coco_data.get('annotations') else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            # imgaug expects polygons as a list of points [[x1, y1], [x2, y2], ...]
            # COCO segmentation for polygons is [x1, y1, x2, y2, ...]
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    # Reshape [x1, y1, x2, y2, ...] to [[x1, y1], [x2, y2], ...]
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points=points, label=ann['category_id']))

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas. Se copiará tal cual.")
        # Add original image and its annotations to the augmented dataset
        augmented_coco_data["images"].append(original_img_info)
        for ann in original_annotations:
            augmented_coco_data["annotations"].append(ann)

        augmented_dataset_info_list.append({
            'id': original_img_info['id'],
            'aligned_image': original_img_info['file_name'],
            'annotation_count': len(original_annotations)
        })
        continue

    print(f"  Procesando original ID {original_img_id} ('{original_file_name}') con {len(segmentations_ia)} segmentaciones.")

    # Add original image and its annotations to the augmented dataset (as the first 'augmented' version, X=0)
    augmented_coco_data["images"].append(original_img_info)
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_info['id'],
        'aligned_image': original_img_info['file_name'],
        'annotation_count': len(original_annotations)
    })

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        # Make a copy of the sequence to ensure different random operations for each augmentation
        seq_det = seq.to_deterministic()

        # Augment image and polygons
        image_aug = seq_det.augment_image(image_original)
        polygons_aug = seq_det.augment_polygons(segmentations_ia)

        # Generate new filenames and IDs
        new_image_filename = f"{Path(original_file_name).stem}_aug{aug_idx}{Path(original_file_name).suffix}"
        current_new_image_id = new_image_id
        new_image_id += 1

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.points) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = poly_ia.points.flatten().tolist()
                x_min, y_min, x_max, y_max = poly_ia.bounding_box.x1,
                                             poly_ia.bounding_box.y1,
                                             poly_ia.bounding_box.x2,
                                             poly_ia.bounding_box.y2
                bbox_aug = [x_min, y_min, x_max - x_min, y_max - y_min]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": poly_ia.area,
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")

**Reasoning**:
The previous code failed due to an `IndentationError` in the multi-line assignment of bounding box coordinates. I will fix this by placing the assignment on a single line to ensure correct indentation.



In [ ]:
ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
new_image_id = max([img['id'] for img in coco_data['images']]) + 1 if coco_data.get('images') else 0
new_annotation_id = max([ann['id'] for ann in coco_data['annotations']]) + 1 if coco_data.get('annotations') else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            # imgaug expects polygons as a list of points [[x1, y1], [x2, y2], ...]
            # COCO segmentation for polygons is [x1, y1, x2, y2, ...]
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    # Reshape [x1, y1, x2, y2, ...] to [[x1, y1], [x2, y2], ...]
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points=points, label=ann['category_id']))

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas. Se copiará tal cual.")
        # Add original image and its annotations to the augmented dataset
        augmented_coco_data["images"].append(original_img_info)
        for ann in original_annotations:
            augmented_coco_data["annotations"].append(ann)

        augmented_dataset_info_list.append({
            'id': original_img_info['id'],
            'aligned_image': original_img_info['file_name'],
            'annotation_count': len(original_annotations)
        })
        continue

    print(f"  Procesando original ID {original_img_id} ('{original_file_name}') con {len(segmentations_ia)} segmentaciones.")

    # Add original image and its annotations to the augmented dataset (as the first 'augmented' version, X=0)
    augmented_coco_data["images"].append(original_img_info)
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_info['id'],
        'aligned_image': original_img_info['file_name'],
        'annotation_count': len(original_annotations)
    })

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        # Make a copy of the sequence to ensure different random operations for each augmentation
        seq_det = seq.to_deterministic()

        # Augment image and polygons
        image_aug = seq_det.augment_image(image_original)
        polygons_aug = seq_det.augment_polygons(segmentations_ia)

        # Generate new filenames and IDs
        new_image_filename = f"{Path(original_file_name).stem}_aug{aug_idx}{Path(original_file_name).suffix}"
        current_new_image_id = new_image_id
        new_image_id += 1

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.points) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = poly_ia.points.flatten().tolist()
                # Fix: ensure bounding box assignment is on a single line
                x_min, y_min, x_max, y_max = poly_ia.bounding_box.x1, poly_ia.bounding_box.y1, poly_ia.bounding_box.x2, poly_ia.bounding_box.y2
                bbox_aug = [x_min, y_min, x_max - x_min, y_max - y_min]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": poly_ia.area,
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")


**Reasoning**:
The `TypeError: Polygon.__init__() got an unexpected keyword argument 'points'` indicates that the `imgaug.Polygon` constructor expects the points as a positional argument, not a keyword argument. I will fix this by passing `points` directly as the first argument.



In [ ]:
ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
new_image_id = max([img['id'] for img in coco_data['images']]) + 1 if coco_data.get('images') else 0
new_annotation_id = max([ann['id'] for ann in coco_data['annotations']]) + 1 if coco_data.get('annotations') else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            # imgaug expects polygons as a list of points [[x1, y1], [x2, y2], ...]
            # COCO segmentation for polygons is [x1, y1, x2, y2, ...]
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    # Reshape [x1, y1, x2, y2, ...] to [[x1, y1], [x2, y2], ...]
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points, label=ann['category_id'])) # FIX: Remove 'points=' keyword

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas. Se copiará tal cual.")
        # Add original image and its annotations to the augmented dataset
        augmented_coco_data["images"].append(original_img_info)
        for ann in original_annotations:
            augmented_coco_data["annotations"].append(ann)

        augmented_dataset_info_list.append({
            'id': original_img_info['id'],
            'aligned_image': original_img_info['file_name'],
            'annotation_count': len(original_annotations)
        })
        continue

    print(f"  Procesando original ID {original_img_id} ('{original_file_name}') con {len(segmentations_ia)} segmentaciones.")

    # Add original image and its annotations to the augmented dataset (as the first 'augmented' version, X=0)
    augmented_coco_data["images"].append(original_img_info)
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_info['id'],
        'aligned_image': original_img_info['file_name'],
        'annotation_count': len(original_annotations)
    })

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        # Make a copy of the sequence to ensure different random operations for each augmentation
        seq_det = seq.to_deterministic()

        # Augment image and polygons
        image_aug = seq_det.augment_image(image_original)
        polygons_aug = seq_det.augment_polygons(segmentations_ia)

        # Generate new filenames and IDs
        new_image_filename = f"{Path(original_file_name).stem}_aug{aug_idx}{Path(original_file_name).suffix}"
        current_new_image_id = new_image_id
        new_image_id += 1

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.points) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = poly_ia.points.flatten().tolist()
                x_min, y_min, x_max, y_max = poly_ia.bounding_box.x1, poly_ia.bounding_box.y1, poly_ia.bounding_box.x2, poly_ia.bounding_box.y2
                bbox_aug = [x_min, y_min, x_max - x_min, y_max - y_min]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": poly_ia.area,
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")

**Reasoning**:
The `ValueError` indicates that `imgaug` needs the image shape to correctly augment polygons. I will provide the `image_original.shape` to the `augment_polygons` method to resolve this.



In [ ]:
ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
new_image_id = max([img['id'] for img in coco_data['images']]) + 1 if coco_data.get('images') else 0
new_annotation_id = max([ann['id'] for ann in coco_data['annotations']]) + 1 if coco_data.get('annotations') else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            # imgaug expects polygons as a list of points [[x1, y1], [x2, y2], ...]
            # COCO segmentation for polygons is [x1, y1, x2, y2, ...]
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    # Reshape [x1, y1, x2, y2, ...] to [[x1, y1], [x2, y2], ...]
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points, label=ann['category_id'])) # FIX: Remove 'points=' keyword

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas. Se copiará tal cual.")
        # Add original image and its annotations to the augmented dataset
        augmented_coco_data["images"].append(original_img_info)
        for ann in original_annotations:
            augmented_coco_data["annotations"].append(ann)

        augmented_dataset_info_list.append({
            'id': original_img_info['id'],
            'aligned_image': original_img_info['file_name'],
            'annotation_count': len(original_annotations)
        })
        continue

    print(f"  Procesando original ID {original_img_id} ('{original_file_name}') con {len(segmentations_ia)} segmentaciones.")

    # Add original image and its annotations to the augmented dataset (as the first 'augmented' version, X=0)
    augmented_coco_data["images"].append(original_img_info)
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_info['id'],
        'aligned_image': original_img_info['file_name'],
        'annotation_count': len(original_annotations)
    })

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        # Make a copy of the sequence to ensure different random operations for each augmentation
        seq_det = seq.to_deterministic()

        # Augment image and polygons
        image_aug = seq_det.augment_image(image_original)
        polygons_aug = seq_det.augment_polygons(segmentations_ia, shape=image_original.shape)

        # Generate new filenames and IDs
        new_image_filename = f"{Path(original_file_name).stem}_aug{aug_idx}{Path(original_file_name).suffix}"
        current_new_image_id = new_image_id
        new_image_id += 1

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.points) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = poly_ia.points.flatten().tolist()
                x_min, y_min, x_max, y_max = poly_ia.bounding_box.x1, poly_ia.bounding_box.y1, poly_ia.bounding_box.x2, poly_ia.bounding_box.y2
                bbox_aug = [x_min, y_min, x_max - x_min, y_max - y_min]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": poly_ia.area,
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")

**Reasoning**:
The previous code failed because `augment_polygons` received an unexpected `shape` argument. The `imgaug` library expects segmentation annotations to be encapsulated in `ia.PolygonsOnImage` objects along with the image shape when augmenting. I will wrap the `ia.Polygon` objects in an `ia.PolygonsOnImage` instance and then pass this to the `augment_polygons` method.



In [ ]:
ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
new_image_id = max([img['id'] for img in coco_data['images']]) + 1 if coco_data.get('images') else 0
new_annotation_id = max([ann['id'] for ann in coco_data['annotations']]) + 1 if coco_data.get('annotations') else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            # imgaug expects polygons as a list of points [[x1, y1], [x2, y2], ...]
            # COCO segmentation for polygons is [x1, y1, x2, y2, ...]
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    # Reshape [x1, y1, x2, y2, ...] to [[x1, y1], [x2, y2], ...]
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points, label=ann['category_id']))

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas. Se copiará tal cual.")
        # Add original image and its annotations to the augmented dataset
        augmented_coco_data["images"].append(original_img_info)
        for ann in original_annotations:
            augmented_coco_data["annotations"].append(ann)

        augmented_dataset_info_list.append({
            'id': original_img_info['id'],
            'aligned_image': original_img_info['file_name'],
            'annotation_count': len(original_annotations)
        })
        continue

    print(f"  Procesando original ID {original_img_id} ('{original_file_name}') con {len(segmentations_ia)} segmentaciones.")

    # Add original image and its annotations to the augmented dataset (as the first 'augmented' version, X=0)
    augmented_coco_data["images"].append(original_img_info)
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_info['id'],
        'aligned_image': original_img_info['file_name'],
        'annotation_count': len(original_annotations)
    })

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        # Make a copy of the sequence to ensure different random operations for each augmentation
        seq_det = seq.to_deterministic()

        # Augment image and polygons
        image_aug = seq_det.augment_image(image_original)

        # FIX: Wrap segmentations_ia in PolygonsOnImage for augmentation
        polygons_on_image = ia.PolygonsOnImage(segmentations_ia, shape=image_original.shape)
        polygons_aug_on_image = seq_det.augment_polygons([polygons_on_image])[0] # Augment and unwrap from batch
        polygons_aug = polygons_aug_on_image.polygons

        # Generate new filenames and IDs
        new_image_filename = f"{Path(original_file_name).stem}_aug{aug_idx}{Path(original_file_name).suffix}"
        current_new_image_id = new_image_id
        new_image_id += 1

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.points) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = poly_ia.points.flatten().tolist()
                x_min, y_min, x_max, y_max = poly_ia.bounding_box.x1, poly_ia.bounding_box.y1, poly_ia.bounding_box.x2, poly_ia.bounding_box.y2
                bbox_aug = [x_min, y_min, x_max - x_min, y_max - y_min]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": poly_ia.area,
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")

**Reasoning**:
The error `AttributeError: 'Polygon' object has no attribute 'points'` indicates that `poly_ia.points` is not the correct way to access the coordinates of an `imgaug.Polygon` object. I will fix this by accessing the x and y coordinates directly using `poly_ia.xx` and `poly_ia.yy` and then combining them into a NumPy array.



In [ ]:
ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
new_image_id = max([img['id'] for img in coco_data['images']]) + 1 if coco_data.get('images') else 0
new_annotation_id = max([ann['id'] for ann in coco_data['annotations']]) + 1 if coco_data.get('annotations') else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            # imgaug expects polygons as a list of points [[x1, y1], [x2, y2], ...]
            # COCO segmentation for polygons is [x1, y1, x2, y2, ...]
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    # Reshape [x1, y1, x2, y2, ...] to [[x1, y1], [x2, y2], ...]
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points, label=ann['category_id']))

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas. Se copiará tal cual.")
        # Add original image and its annotations to the augmented dataset
        augmented_coco_data["images"].append(original_img_info)
        for ann in original_annotations:
            augmented_coco_data["annotations"].append(ann)

        augmented_dataset_info_list.append({
            'id': original_img_info['id'],
            'aligned_image': original_img_info['file_name'],
            'annotation_count': len(original_annotations)
        })
        continue

    print(f"  Procesando original ID {original_img_id} ('{original_file_name}') con {len(segmentations_ia)} segmentaciones.")

    # Add original image and its annotations to the augmented dataset (as the first 'augmented' version, X=0)
    augmented_coco_data["images"].append(original_img_info)
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_info['id'],
        'aligned_image': original_img_info['file_name'],
        'annotation_count': len(original_annotations)
    })

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        # Make a copy of the sequence to ensure different random operations for each augmentation
        seq_det = seq.to_deterministic()

        # Augment image and polygons
        image_aug = seq_det.augment_image(image_original)

        # FIX: Wrap segmentations_ia in PolygonsOnImage for augmentation
        polygons_on_image = ia.PolygonsOnImage(segmentations_ia, shape=image_original.shape)
        polygons_aug_on_image = seq_det.augment_polygons([polygons_on_image])[0] # Augment and unwrap from batch
        polygons_aug = polygons_aug_on_image.polygons

        # Generate new filenames and IDs
        new_image_filename = f"{Path(original_file_name).stem}_aug{aug_idx}{Path(original_file_name).suffix}"
        current_new_image_id = new_image_id
        new_image_id += 1

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            # FIX: Access points using poly_ia.xx and poly_ia.yy instead of poly_ia.points
            # Also adjust the check for minimum points accordingly
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.xx) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = np.column_stack([poly_ia.xx, poly_ia.yy]).flatten().tolist()
                x_min, y_min, x_max, y_max = poly_ia.bounding_box.x1, poly_ia.bounding_box.y1, poly_ia.bounding_box.x2, poly_ia.bounding_box.y2
                bbox_aug = [x_min, y_min, x_max - x_min, y_max - y_min]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": poly_ia.area,
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")


**Reasoning**:
The previous code failed because `imgaug.Polygon` objects do not have a `bounding_box` attribute directly. I need to call the `to_bounding_box()` method on the polygon object to obtain its bounding box and then extract the coordinates (`x1`, `y1`, `x2`, `y2`) from the resulting `BoundingBox` object.



In [ ]:
ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
new_image_id = max([img['id'] for img in coco_data['images']]) + 1 if coco_data.get('images') else 0
new_annotation_id = max([ann['id'] for ann in coco_data['annotations']]) + 1 if coco_data.get('annotations') else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            # imgaug expects polygons as a list of points [[x1, y1], [x2, y2], ...]
            # COCO segmentation for polygons is [x1, y1, x2, y2, ...]
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    # Reshape [x1, y1, x2, y2, ...] to [[x1, y1], [x2, y2], ...]
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points, label=ann['category_id']))

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas. Se copiará tal cual.")
        # Add original image and its annotations to the augmented dataset
        augmented_coco_data["images"].append(original_img_info)
        for ann in original_annotations:
            augmented_coco_data["annotations"].append(ann)

        augmented_dataset_info_list.append({
            'id': original_img_info['id'],
            'aligned_image': original_img_info['file_name'],
            'annotation_count': len(original_annotations)
        })
        continue

    print(f"  Procesando original ID {original_img_id} ('{original_file_name}') con {len(segmentations_ia)} segmentaciones.")

    # Add original image and its annotations to the augmented dataset (as the first 'augmented' version, X=0)
    augmented_coco_data["images"].append(original_img_info)
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_info['id'],
        'aligned_image': original_img_info['file_name'],
        'annotation_count': len(original_annotations)
    })

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        # Make a copy of the sequence to ensure different random operations for each augmentation
        seq_det = seq.to_deterministic()

        # Augment image and polygons
        image_aug = seq_det.augment_image(image_original)

        # FIX: Wrap segmentations_ia in PolygonsOnImage for augmentation
        polygons_on_image = ia.PolygonsOnImage(segmentations_ia, shape=image_original.shape)
        polygons_aug_on_image = seq_det.augment_polygons([polygons_on_image])[0] # Augment and unwrap from batch
        polygons_aug = polygons_aug_on_image.polygons

        # Generate new filenames and IDs
        new_image_filename = f"{Path(original_file_name).stem}_aug{aug_idx}{Path(original_file_name).suffix}"
        current_new_image_id = new_image_id
        new_image_id += 1

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            # FIX: Access points using poly_ia.xx and poly_ia.yy instead of poly_ia.points
            # Also adjust the check for minimum points accordingly
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.xx) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = np.column_stack([poly_ia.xx, poly_ia.yy]).flatten().tolist()
                # FIX: Get bounding box from the polygon object
                bbox_obj = poly_ia.to_bounding_box()
                x_min, y_min, x_max, y_max = bbox_obj.x1, bbox_obj.y1, bbox_obj.x2, bbox_obj.y2
                bbox_aug = [x_min, y_min, x_max - x_min, y_max - y_min]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": poly_ia.area,
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")


**Reasoning**:
The previous code failed with a `TypeError: Object of type float32 is not JSON serializable` when trying to save the `augmented_coco_data` to JSON. This means some numerical values (likely from bounding boxes or segmentations) are still in NumPy's `float32` format. I need to convert these `float32` values to standard Python floats before dumping the JSON data.



In [ ]:
ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
new_image_id = max([img['id'] for img in coco_data['images']]) + 1 if coco_data.get('images') else 0
new_annotation_id = max([ann['id'] for ann in coco_data['annotations']]) + 1 if coco_data.get('annotations') else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            # imgaug expects polygons as a list of points [[x1, y1], [x2, y2], ...]
            # COCO segmentation for polygons is [x1, y1, x2, y2, ...]
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    # Reshape [x1, y1, x2, y2, ...] to [[x1, y1], [x2, y2], ...]
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points, label=ann['category_id']))

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas. Se copiará tal cual.")
        # Add original image and its annotations to the augmented dataset
        augmented_coco_data["images"].append(original_img_info)
        for ann in original_annotations:
            augmented_coco_data["annotations"].append(ann)

        augmented_dataset_info_list.append({
            'id': original_img_info['id'],
            'aligned_image': original_img_info['file_name'],
            'annotation_count': len(original_annotations)
        })
        continue

    print(f"  Procesando original ID {original_img_id} ('{original_file_name}') con {len(segmentations_ia)} segmentaciones.")

    # Add original image and its annotations to the augmented dataset (as the first 'augmented' version, X=0)
    augmented_coco_data["images"].append(original_img_info)
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_info['id'],
        'aligned_image': original_img_info['file_name'],
        'annotation_count': len(original_annotations)
    })

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        # Make a copy of the sequence to ensure different random operations for each augmentation
        seq_det = seq.to_deterministic()

        # Augment image and polygons
        image_aug = seq_det.augment_image(image_original)

        # FIX: Wrap segmentations_ia in PolygonsOnImage for augmentation
        polygons_on_image = ia.PolygonsOnImage(segmentations_ia, shape=image_original.shape)
        polygons_aug_on_image = seq_det.augment_polygons([polygons_on_image])[0] # Augment and unwrap from batch
        polygons_aug = polygons_aug_on_image.polygons

        # Generate new filenames and IDs
        new_image_filename = f"{Path(original_file_name).stem}_aug{aug_idx}{Path(original_file_name).suffix}"
        current_new_image_id = new_image_id
        new_image_id += 1

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            # FIX: Access points using poly_ia.xx and poly_ia.yy instead of poly_ia.points
            # Also adjust the check for minimum points accordingly
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.xx) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = np.column_stack([poly_ia.xx, poly_ia.yy]).flatten().tolist()
                # Ensure all elements in coco_segmentation are standard Python floats
                coco_segmentation = [float(p) for p in coco_segmentation]

                # FIX: Get bounding box from the polygon object
                bbox_obj = poly_ia.to_bounding_box()
                x_min, y_min, x_max, y_max = bbox_obj.x1, bbox_obj.y1, bbox_obj.x2, bbox_obj.y2
                bbox_aug = [float(x_min), float(y_min), float(x_max - x_min), float(y_max - y_min)]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": float(poly_ia.area),
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")


## Update Training Script for Augmented Data

### Subtask:
Modify the `aligned_dir` variable in the existing training script (cell `KH6EzZzmVYGT`) to point to the newly created directory containing the augmented dataset.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import json

# ----------------------------
# 1. DATASET PARA ENTRENAMIENTO
# ----------------------------
class AlopeciaDataset(Dataset):
    def __init__(self, aligned_dir, img_size=256):
        self.img_dir = os.path.join(aligned_dir, 'images')
        self.annot_dir = os.path.join(aligned_dir, 'annotations')
        self.img_size = img_size

        # Cargar información del dataset
        info_path = os.path.join(aligned_dir, 'annotations', 'dataset_info.json')
        with open(info_path, 'r') as f:
            self.dataset_info = json.load(f)

        # También cargar COCO filtrado
        coco_path = os.path.join(aligned_dir, 'annotations', 'filtered_coco.json')
        with open(coco_path, 'r') as f:
            self.coco_data = json.load(f)

        print(f"Dataset cargado: {len(self.dataset_info)} imágenes, {sum(item['annotation_count'] for item in self.dataset_info)} anotaciones")

    def __len__(self):
        return len(self.dataset_info)

    def __getitem__(self, idx):
        item = self.dataset_info[idx]
        img_path = os.path.join(self.img_dir, item['aligned_image'])

        # Cargar imagen
        img = cv2.imread(img_path)
        img = cv2.resize(img, (self.img_size, self.img_size))
        img = img / 255.0  # Normalizar [0, 1]
        img = torch.tensor(img).permute(2, 0, 1).float()

        # Crear máscara
        mask = self._create_mask_from_coco(item['id'], self.img_size)
        mask = torch.tensor(mask).unsqueeze(0).float()

        return img, mask

    def _create_mask_from_coco(self, image_id, size):
        mask = np.zeros((size, size), dtype=np.uint8)

        # Get image info
        image_info = None
        for img_info in self.coco_data['images']:
            if img_info['id'] == image_id:
                image_info = img_info
                break
        if image_info is None:
            raise ValueError(f"Image info not found for image_id: {image_id}")

        img_width = image_info['width']
        img_height = image_info['height']

        # Buscar anotaciones
        for ann in self.coco_data['annotations']:
            if ann['image_id'] == image_id:
                seg = ann['segmentation']
                if isinstance(seg, list):
                    for polygon in seg:
                        pts = np.array(polygon).reshape(-1, 2)
                        pts = (pts * size / max(img_width, img_height)).astype(np.int32)
                        cv2.fillPoly(mask, [pts], 1)

        return mask

# ----------------------------
# 2. MODELO MEJORADO (Sin Sigmoid al final)
# ----------------------------
class SimpleUNet(nn.Module):
    def __init__(self):
        super(SimpleUNet, self).__init__()

        # Encoder
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )

        # Decoder
        self.up1 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(192, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU()
        )
        self.up2 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(67, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU()
        )

        # Salida (logits, sin sigmoid)
        self.output = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        b = self.bottleneck(e2)

        d1 = self.up1(b)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)

        d2 = self.up2(d1)
        d2 = torch.cat([d2, x[:, :, :d2.shape[2], :d2.shape[3]]], dim=1)
        d2 = self.dec2(d2)

        out = self.output(d2)
        return out # <-- Dejamos los logits crudos

# ----------------------------
# 3. ENTRENAMIENTO (Con pesos para clases desbalanceadas)
# ----------------------------
def train_model():
    aligned_dir = '/content/alopecia_dataset/augmented_dataset'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Usando dispositivo: {device}")

    dataset = AlopeciaDataset(aligned_dir, img_size=256)
    dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

    model = SimpleUNet().to(device)

    # PESO PARA LA CLASE POSITIVA (ALOPECIA)
    # Esto penaliza 10 veces más equivocarse en la alopecia que en el fondo
    pos_weight = torch.tensor([10.0]).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 30
    train_losses = []

    print("\n=== COMIENZO DEL ENTRENAMIENTO MEJORADO ===")

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0

        for batch_idx, (images, masks) in enumerate(dataloader):
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            loss = criterion(outputs, masks)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            if batch_idx % 2 == 0:
                 print(f"  Batch {batch_idx}: Loss = {loss.item():.4f}")

        avg_loss = epoch_loss / len(dataloader)
        train_losses.append(avg_loss)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

        if (epoch + 1) % 5 == 0:
            checkpoint_path = f'/content/alopecia_dataset/model_checkpoint_epoch_{epoch+1}.pth'
            torch.save(model.state_dict(), checkpoint_path)
            print(f"  Checkpoint guardado: {checkpoint_path}")

    final_model_path = '/content/alopecia_dataset/alopecia_segmentation_model.pth'
    torch.save(model.state_dict(), final_model_path)
    print(f"\n✅ Modelo final guardado: {final_model_path}")

    return model, train_losses

# ----------------------------
# 4. VISUALIZACIÓN MEJORADA (Mapas de calor)
# ----------------------------
def visualize_predictions(model, dataset, num_samples=3):
    model.eval()
    device = next(model.parameters()).device

    # Visualizamos 3 filas, 4 columnas
    fig, axes = plt.subplots(num_samples, 4, figsize=(16, 4*num_samples))

    for i in range(num_samples):
        img, true_mask = dataset[i]
        img_tensor = img.unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model(img_tensor)
            probs = torch.sigmoid(logits) # Convertimos logits a prob (0-1)
            pred_mask = (probs > 0.3).float() # Umbral bajado a 0.3

        img_np = img.permute(1, 2, 0).cpu().numpy()
        true_mask_np = true_mask[0].cpu().numpy()
        probs_np = probs[0][0].cpu().numpy()
        pred_mask_np = pred_mask[0][0].cpu().numpy()

        # Imagen
        axes[i, 0].imshow(img_np)
        axes[i, 0].set_title(f"Imagen {i+1}")
        axes[i, 0].axis('off')

        # Real
        axes[i, 1].imshow(true_mask_np, cmap='gray')
        axes[i, 1].set_title("Real")
        axes[i, 1].axis('off')

        # Calor (Probabilidades)
        im = axes[i, 2].imshow(probs_np, cmap='jet', vmin=0, vmax=1)
        axes[i, 2].set_title("Confianza (Mapa Calor)")
        axes[i, 2].axis('off')
        plt.colorbar(im, ax=axes[i, 2], fraction=0.046, pad=0.04)

        # Predicción binaria
        axes[i, 3].imshow(pred_mask_np, cmap='gray')
        axes[i, 3].set_title("Predicción (>0.3)")
        axes[i, 3].axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    print("🚀 INICIANDO ENTRENAMIENTO DEL MODELO DE SEGMENTACIÓN")
    try:
        trained_model, losses = train_model()

        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.plot(losses)
        plt.xlabel('Época')
        plt.ylabel('Pérdida')
        plt.title('Curva de Aprendizaje')
        plt.grid(True)

        aligned_dir = '/content/alopecia_dataset/augmented_dataset'
        dataset = AlopeciaDataset(aligned_dir, img_size=256)

        print("\\n🎯 VISUALIZANDO PREDICCIONES DEL MODELO")
        visualize_predictions(trained_model, dataset, num_samples=min(3, len(dataset)))

        print("\\n✅ ¡ENTRENAMIENTO COMPLETADO!")
    except Exception as e:
        print(f"❌ Error durante la ejecución: {e}")
        import traceback
        traceback.print_exc()


**Reasoning**:
The previous code failed because `cv2.imread` returned `None` due to image files not being present in the `augmented_img_dir` for all entries in the dataset. This happened because original images were not always explicitly copied to the augmented directory, especially if they had no segmentation annotations. The fix involves ensuring that every original image that is processed (regardless of annotations) is copied to the `augmented_img_dir` with a consistent filename (based on its COCO ID) and that its corresponding entry in `augmented_coco_data` and `augmented_dataset_info_list` reflects this filename. The augmented images will also use a consistent naming convention based on the original COCO ID.



In [ ]:
import os
import json
import shutil
from pathlib import Path
import cv2
import numpy as np
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

# ================= RELOAD NECESSARY DATA FROM PREVIOUS CELLS =================
# These variables were defined in cell TPUKkd36GGgz and might have been lost
# after runtime restart or numpy downgrade affecting the environment.

# Explicitly define `json_path` and `images_dir` based on prior execution's output
json_path = '/content/temp_extract/test-coco/result_coco.json'
images_dir = '/content/temp_extract/test-coco/images'
extract_dir = '/content/temp_extract'

# Load `coco_data`
with open(json_path, 'r') as f:
    coco_data = json.load(f)
print(f"✅ `coco_data` re-loaded from {json_path}.")

# Re-list `imagenes` (image paths found during extraction)
imagenes = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.PNG']:
    imagenes.extend(list(Path(images_dir).rglob(ext)))
print(f"✅ `imagenes` re-listed: {len(imagenes)} found in {images_dir}.")

# ================= 1. DEFINIR DIRECTORIOS Y CONFIGURACIÓN =================
# Output: Directorio para el dataset aumentado
augmented_output_dir = Path('/content/alopecia_dataset/augmented_dataset')
augmented_img_dir = augmented_output_dir / 'images'
augmented_annot_dir = augmented_output_dir / 'annotations'

# Limpiar y crear directorios
shutil.rmtree(augmented_output_dir, ignore_errors=True)
augmented_img_dir.mkdir(parents=True, exist_ok=True)
augmented_annot_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Directorios para el dataset aumentado creados en: {augmented_output_dir}")

# ================= 2. INICIALIZAR AUMENTADORES imgaug =================
# Definir la secuencia de aumentación. Usamos `random_order=True` para variar el orden.
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # Volteo horizontal con 50% de probabilidad
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # Escala 80-120%
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # Traslación -20% a +20%
        rotate=(-25, 25), # Rotación -25 a +25 grados
        shear=(-8, 8) # Cizallamiento -8 a +8 grados
    ),
    iaa.Multiply((0.8, 1.2), per_channel=0.2), # Ajuste de brillo (80-120%)
    iaa.LinearContrast((0.75, 1.5)), # Ajuste de contraste
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), # Ruido Gaussiano
    iaa.Sometimes(0.2, iaa.Sharpen(alpha=(0.0, 1.0), lightness=(0.75, 1.5))), # Afilado
    iaa.Sometimes(0.2, iaa.Emboss(alpha=(0.0, 1.0), strength=(0.5, 1.5))), # Relieve
    iaa.Sometimes(0.2, iaa.EdgeDetect(alpha=(0.0, 0.7))), # Detección de bordes
    iaa.Sometimes(0.2, iaa.MotionBlur(k=(3, 7), angle=[-45, 45])), # Desenfoque de movimiento
    iaa.Sometimes(0.2, iaa.AddToHueAndSaturation((-20, 20))) # Cambios de tono y saturación
], random_order=True) # Aplicar aumentaciones en orden aleatorio

print("✅ Secuencia de aumentación de imgaug inicializada.")

# ================= 3. & 4. CARGAR COCO ORIGINAL Y MAPPING =================
# `coco_data` y `imagenes` ya están disponibles desde la ejecución anterior.
# Asegurémonos de tener un mapa de image_id a info de imagen y anotaciones

original_images_info = {img['id']: img for img in coco_data.get('images', [])}
original_annotations_info = {}
for ann in coco_data.get('annotations', []):
    image_id = ann['image_id']
    if image_id not in original_annotations_info:
        original_annotations_info[image_id] = []
    original_annotations_info[image_id].append(ann)

print(f"✅ COCO original cargado. Imágenes: {len(original_images_info)}, Anotaciones para {len(original_annotations_info)} imágenes.")

ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
# Starting new_image_id and new_annotation_id from highest existing ID + 1
existing_image_ids = [img['id'] for img in coco_data.get('images', [])]
new_image_id = max(existing_image_ids) + 1 if existing_image_ids else 0

existing_annotation_ids = [ann['id'] for ann in coco_data.get('annotations', [])]
new_annotation_id = max(existing_annotation_ids) + 1 if existing_annotation_ids else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada en `imagenes`. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB) # Convert to RGB for imgaug

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points, label=ann['category_id']))

    # --- Handle the original image itself (always copy if successfully loaded) ---
    # Use original COCO image ID as the basis for the filename
    original_image_saved_name = f"{original_img_id}{original_img_path.suffix}"
    cv2.imwrite(str(augmented_img_dir / original_image_saved_name), cv2.cvtColor(image_original, cv2.COLOR_RGB2BGR))

    # Create a new image info dictionary for the COCO format, ensuring file_name points to the copied file
    original_img_coco_entry = original_img_info.copy() # Make a copy to avoid modifying original_images_info
    original_img_coco_entry['file_name'] = original_image_saved_name # Update filename to match the saved one
    augmented_coco_data["images"].append(original_img_coco_entry)

    # Add original annotations
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_id, # Use original ID for this entry
        'aligned_image': original_image_saved_name, # This is the key for AlopeciaDataset
        'annotation_count': len(original_annotations)
    })
    print(f"  Añadida imagen original ID {original_img_id} ('{original_file_name}') a dataset aumentado.")

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas para aumentación. Solo se añadió la versión original.")
        continue # Skip augmentation for this image if no valid segmentations

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        seq_det = seq.to_deterministic()
        image_aug = seq_det.augment_image(image_original)

        # Wrap segmentations_ia in PolygonsOnImage for augmentation
        polygons_on_image = ia.PolygonsOnImage(segmentations_ia, shape=image_original.shape)
        polygons_aug_on_image = seq_det.augment_polygons([polygons_on_image])[0] # Augment and unwrap from batch
        polygons_aug = polygons_aug_on_image.polygons

        # Generate new filenames and IDs
        # Ensure augmented IDs are distinct from original IDs and from each other
        current_new_image_id = new_image_id
        new_image_id += 1 # Increment for the next augmented image
        new_image_filename = f"{original_img_id}_aug{aug_idx}{original_img_path.suffix}" # Use original COCO ID as base

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry for augmented image
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.xx) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = np.column_stack([poly_ia.xx, poly_ia.yy]).flatten().tolist()
                # Ensure all elements in coco_segmentation are standard Python floats
                coco_segmentation = [float(p) for p in coco_segmentation]

                # Get bounding box from the polygon object
                bbox_obj = poly_ia.to_bounding_box()
                x_min, y_min, x_max, y_max = bbox_obj.x1, bbox_obj.y1, bbox_obj.x2, bbox_obj.y2
                bbox_aug = [float(x_min), float(y_min), float(x_max - x_min), float(y_max - y_min)]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": float(poly_ia.area), # Ensure JSON serializable float
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1 # Increment for the next annotation
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")

**Reasoning**:
The previous errors in data augmentation were resolved, and the augmented dataset generation completed successfully. Now, I will re-execute the training script in cell `KH6EzZzmVYGT` with `aligned_dir` pointing to the newly generated augmented dataset to ensure it runs without `cv2.resize` assertion errors.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import json

# ----------------------------
# 1. DATASET PARA ENTRENAMIENTO
# ----------------------------
class AlopeciaDataset(Dataset):
    def __init__(self, aligned_dir, img_size=256):
        self.img_dir = os.path.join(aligned_dir, 'images')
        self.annot_dir = os.path.join(aligned_dir, 'annotations')
        self.img_size = img_size

        # Cargar información del dataset
        info_path = os.path.join(aligned_dir, 'annotations', 'dataset_info.json')
        with open(info_path, 'r') as f:
            self.dataset_info = json.load(f)

        # También cargar COCO filtrado
        coco_path = os.path.join(aligned_dir, 'annotations', 'filtered_coco.json')
        with open(coco_path, 'r') as f:
            self.coco_data = json.load(f)

        print(f"Dataset cargado: {len(self.dataset_info)} imágenes, {sum(item['annotation_count'] for item in self.dataset_info)} anotaciones")

    def __len__(self):
        return len(self.dataset_info)

    def __getitem__(self, idx):
        item = self.dataset_info[idx]
        img_path = os.path.join(self.img_dir, item['aligned_image'])

        # Cargar imagen
        img = cv2.imread(img_path)
        img = cv2.resize(img, (self.img_size, self.img_size))
        img = img / 255.0  # Normalizar [0, 1]
        img = torch.tensor(img).permute(2, 0, 1).float()

        # Crear máscara
        mask = self._create_mask_from_coco(item['id'], self.img_size)
        mask = torch.tensor(mask).unsqueeze(0).float()

        return img, mask

    def _create_mask_from_coco(self, image_id, size):
        mask = np.zeros((size, size), dtype=np.uint8)

        # Get image info
        image_info = None
        for img_info in self.coco_data['images']:
            if img_info['id'] == image_id:
                image_info = img_info
                break
        if image_info is None:
            raise ValueError(f"Image info not found for image_id: {image_id}")

        img_width = image_info['width']
        img_height = image_info['height']

        # Buscar anotaciones
        for ann in self.coco_data['annotations']:
            if ann['image_id'] == image_id:
                seg = ann['segmentation']
                if isinstance(seg, list):
                    for polygon in seg:
                        pts = np.array(polygon).reshape(-1, 2)
                        pts = (pts * size / max(img_width, img_height)).astype(np.int32)
                        cv2.fillPoly(mask, [pts], 1)

        return mask

# ----------------------------
# 2. MODELO MEJORADO (Sin Sigmoid al final)
# ----------------------------
class SimpleUNet(nn.Module):
    def __init__(self):
        super(SimpleUNet, self).__init__()

        # Encoder
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )

        # Decoder
        self.up1 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(192, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU()
        )
        self.up2 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(67, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU()
        )

        # Salida (logits, sin sigmoid)
        self.output = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        b = self.bottleneck(e2)

        d1 = self.up1(b)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)

        d2 = self.up2(d1)
        d2 = torch.cat([d2, x[:, :, :d2.shape[2], :d2.shape[3]]], dim=1)
        d2 = self.dec2(d2)

        out = self.output(d2)
        return out # <-- Dejamos los logits crudos

# ----------------------------
# 3. ENTRENAMIENTO (Con pesos para clases desbalanceadas)
# ----------------------------
def train_model():
    aligned_dir = '/content/alopecia_dataset/augmented_dataset'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Usando dispositivo: {device}")

    dataset = AlopeciaDataset(aligned_dir, img_size=256)
    dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

    model = SimpleUNet().to(device)

    # PESO PARA LA CLASE POSITIVA (ALOPECIA)
    # Esto penaliza 10 veces más equivocarse en la alopecia que en el fondo
    pos_weight = torch.tensor([10.0]).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 30
    train_losses = []

    print("\n=== COMIENZO DEL ENTRENAMIENTO MEJORADO ===")

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0

        for batch_idx, (images, masks) in enumerate(dataloader):
            images = images.to(device)
            masks = masks.to(device)

            outputs = model(images)
            loss = criterion(outputs, masks)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            if batch_idx % 2 == 0:
                 print(f"  Batch {batch_idx}: Loss = {loss.item():.4f}")

        avg_loss = epoch_loss / len(dataloader)
        train_losses.append(avg_loss)

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

        if (epoch + 1) % 5 == 0:
            checkpoint_path = f'/content/alopecia_dataset/model_checkpoint_epoch_{epoch+1}.pth'
            torch.save(model.state_dict(), checkpoint_path)
            print(f"  Checkpoint guardado: {checkpoint_path}")

    final_model_path = '/content/alopecia_dataset/alopecia_segmentation_model.pth'
    torch.save(model.state_dict(), final_model_path)
    print(f"\n✅ Modelo final guardado: {final_model_path}")

    return model, train_losses

# ----------------------------
# 4. VISUALIZACIÓN MEJORADA (Mapas de calor)
# ----------------------------
def visualize_predictions(model, dataset, num_samples=3):
    model.eval()
    device = next(model.parameters()).device

    # Visualizamos 3 filas, 4 columnas
    fig, axes = plt.subplots(num_samples, 4, figsize=(16, 4*num_samples))

    for i in range(num_samples):
        img, true_mask = dataset[i]
        img_tensor = img.unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model(img_tensor)
            probs = torch.sigmoid(logits) # Convertimos logits a prob (0-1)
            pred_mask = (probs > 0.3).float() # Umbral bajado a 0.3

        img_np = img.permute(1, 2, 0).cpu().numpy()
        true_mask_np = true_mask[0].cpu().numpy()
        probs_np = probs[0][0].cpu().numpy()
        pred_mask_np = pred_mask[0][0].cpu().numpy()

        # Imagen
        axes[i, 0].imshow(img_np)
        axes[i, 0].set_title(f"Imagen {i+1}")
        axes[i, 0].axis('off')

        # Real
        axes[i, 1].imshow(true_mask_np, cmap='gray')
        axes[i, 1].set_title("Real")
        axes[i, 1].axis('off')

        # Calor (Probabilidades)
        im = axes[i, 2].imshow(probs_np, cmap='jet', vmin=0, vmax=1)
        axes[i, 2].set_title("Confianza (Mapa Calor)")
        axes[i, 2].axis('off')
        plt.colorbar(im, ax=axes[i, 2], fraction=0.046, pad=0.04)

        # Predicción binaria
        axes[i, 3].imshow(pred_mask_np, cmap='gray')
        axes[i, 3].set_title("Predicción (>0.3)")
        axes[i, 3].axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    print("✨ INICIANDO ENTRENAMIENTO DEL MODELO DE SEGMENTACIÓN")
    try:
        trained_model, losses = train_model()

        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.plot(losses)
        plt.xlabel('Época')
        plt.ylabel('Pérdida')
        plt.title('Curva de Aprendizaje')
        plt.grid(True)

        aligned_dir = '/content/alopecia_dataset/augmented_dataset'
        dataset = AlopeciaDataset(aligned_dir, img_size=256)

        print("\\n🎯 VISUALIZANDO PREDICCIONES DEL MODELO")
        visualize_predictions(trained_model, dataset, num_samples=min(3, len(dataset)))

        print("\\n✅ ¡ENTRENAMIENTO COMPLETADO!")
    except Exception as e:
        print(f"❌ Error durante la ejecución: {e}")
        import traceback
        traceback.print_exc()


## Final Task

### Subtask:
Summarize the entire process, confirming the successful generation of synthetic data, the training of the segmentation model with this augmented dataset, and any observed improvements or next steps.


# Task
**Summary of the Refactoring Process and Model Training:**

The entire process of setting up, preparing data, augmenting, training, and testing a segmentation model for alopecia detection has been successfully refactored and executed.

Here's a breakdown of the key steps and their successful completion:

1.  **Consolidated Initial Setup and Data Extraction**:
    *   Google Drive was successfully mounted.
    *   The COCO dataset ZIP file (`test-coco-upload.zip`) was extracted to a temporary directory (`/content/temp_extract`).
    *   COCO JSON annotations (`/content/temp_extract/test-coco/result_coco.json`) and the list of image paths were loaded, initializing `coco_data`, `imagenes`, `json_path`, `images_dir`, and `extract_dir` for subsequent steps.

2.  **Consolidated Library Installations and NumPy Downgrade**:
    *   Necessary Python libraries (`imgaug`, `opencv-python`, `scikit-image`) were installed.
    *   A critical `numpy` downgrade to `1.26.4` was successfully performed to ensure compatibility with `imgaug`, resolving `AttributeError` related to `np.sctypes`.

3.  **Refactored and Generated Augmented Segmentation Dataset**:
    *   The code for generating synthetic images and their COCO annotations was refactored.
    *   The process generated **110 images** and **180 annotations** in the `augmented_dataset` directory (`/content/alopecia_dataset/augmented_dataset`).
    *   This involved applying various geometric and photometric transformations using `imgaug` to the original dataset.
    *   Several issues encountered during augmentation were successfully resolved:
        *   `NameError` for `coco_data` and `imagenes` was fixed by explicitly reloading these variables.
        *   `IndentationError` in bounding box assignment was corrected.
        *   `TypeError` when creating `ia.Polygon` was resolved by passing points as a positional argument.
        *   `ValueError` related to missing image shape for `augment_polygons` was fixed by wrapping `ia.Polygon` objects in `ia.PolygonsOnImage` with the correct image shape.
        *   `AttributeError` for `poly_ia.points` and `poly_ia.bounding_box` was resolved by using `poly_ia.xx`, `poly_ia.yy`, and `poly_ia.to_bounding_box()`.
        *   `TypeError: Object of type float32 is not JSON serializable` was fixed by explicitly converting all numerical values (segmentation coordinates, bbox coordinates, area) to standard Python floats before JSON serialization.
        *   `cv2.resize` assertion error was fixed by ensuring that all original images are copied to the `augmented_dataset` directory with consistent naming (using their COCO IDs) before augmentation, guaranteeing that all dataset entries point to existing image files.
    *   The augmented images and their corresponding updated COCO JSON files (`filtered_coco.json`, `dataset_info.json`) were correctly saved in the `augmented_dataset` directory.

4.  **Refactored and Trained Segmentation Model with Augmented Data**:
    *   The `AlopeciaDataset` class, `SimpleUNet` model definition, and the training loop were consolidated and refactored.
    *   The `SimpleUNet` model was successfully trained for **30 epochs** using the newly generated augmented dataset (`/content/alopecia_dataset/augmented_dataset`).
    *   Checkpoints were saved every 5 epochs, and the final trained model (`/content/alopecia_dataset/alopecia_segmentation_model.pth`) was stored.
    *   The training process now correctly loads images and masks from the augmented dataset, and the loss curve indicates the model's learning progression.

5.  **Refactored and Visualized Model Predictions**:
    *   The code for visualizing model predictions was refactored and successfully executed.
    *   It now correctly loads the trained segmentation model, preprocesses a new image (from the extracted original dataset), performs prediction, and visualizes the original image, probability map, and binary mask.

**Conclusion**:
All components of the project, from initial data setup and augmentation to model training and testing, have been successfully executed and refactored. The generation of augmented data significantly increased the dataset size from 12 original images to 110 images, providing a more robust dataset for training the segmentation model. The model trained effectively on this expanded dataset, and its predictions can now be visualized. The next steps would involve further evaluation, potential hyperparameter tuning, and deployment considerations based on the model's performance.

## Consolidate Initial Setup and Data Extraction

### Subtask:
Merge and refactor all initial setup steps, including mounting Google Drive, extracting the COCO dataset ZIP file to a temporary directory, and loading the COCO JSON annotations along with the list of found image paths. This step will ensure that `coco_data`, `imagenes`, `json_path`, `images_dir`, and `extract_dir` are correctly initialized and accessible for subsequent steps in a clean manner.


**Reasoning**:
I need to combine all initial setup and data extraction steps into a single code block as requested. This includes importing libraries, mounting Drive, defining paths, extracting the ZIP, finding JSON and image directories, listing image files, and loading the COCO JSON data. This ensures all required variables are correctly initialized and accessible.



In [ ]:
from google.colab import drive
import os, json, shutil, zipfile
from pathlib import Path
import glob # Import glob for searching zip files

# ================= 1. MONTAR GOOGLE DRIVE =================
drive.mount('/content/drive')

# ================= 2. CONFIGURACIÓN =================
# Ruta al ZIP en Google Drive (¡AJUSTA ESTA RUTA SI ES NECESARIO!)
zip_path = '/content/drive/MyDrive/alopecia_project/data/test-coco-upload.zip'

# Ruta donde quieres guardar el dataset YOLO en Drive (se usará más adelante)
output_dir = '/content/drive/MyDrive/alopecia_project/yolo_dataset'

# ================= 3. EXTRAER ZIP =================
print("📦 Extrayendo ZIP de Google Drive...")
extract_dir = '/content/temp_extract'
shutil.rmtree(extract_dir, ignore_errors=True)
os.makedirs(extract_dir)

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        print("Contenido del ZIP:")
        file_list = zip_ref.namelist()
        for file in file_list[:5]: # Mostrar primeros 5
            print(f"  - {file}")
        if len(file_list) > 5:
            print(f"  ... y {len(file_list) - 5} más")
        zip_ref.extractall(extract_dir)
    print(f"✅ ZIP extraído en: {extract_dir}")

except FileNotFoundError:
    print(f"❌ No se encontró el ZIP en: {zip_path}")
    print("\n📁 Buscando archivos ZIP en Google Drive...")
    zip_files = glob.glob('/content/drive/MyDrive/**/*.zip', recursive=True)
    if zip_files:
        print("Archivos ZIP encontrados:")
        for zf in zip_files[:5]:
            print(f"  - {zf}")
        # Usar el primer ZIP encontrado como predeterminado
        zip_path = zip_files[0]
        print(f"\n✅ Usando: {zip_path}")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"✅ ZIP extraído en: {extract_dir}")
    else:
        raise FileNotFoundError("No se encontró ningún archivo ZIP en Google Drive.")

# ================= 4. BUSCAR ARCHIVOS JSON Y CARPETA DE IMÁGENES =================
# Buscar JSON
json_files = list(Path(extract_dir).rglob('*.json'))
if not json_files:
    raise FileNotFoundError("❌ No se encontró ningún archivo JSON en el ZIP extraído.")
json_path = str(json_files[0])
print(f"✅ JSON encontrado: {json_path}")

# Buscar carpeta de imágenes
images_dir = None
possible_img_subdirs = ['images', 'img', 'upload', 'data', 'media']
# First, check common subdirectories within the folder containing the JSON
json_parent_dir = Path(json_path).parent
for sub_dir_name in possible_img_subdirs:
    candidate_images_dir = json_parent_dir / sub_dir_name
    if candidate_images_dir.is_dir() and any(f.suffix.lower() in ['.jpg', '.png', '.jpeg', '.webp'] for f in candidate_images_dir.iterdir()):
        images_dir = candidate_images_dir
        break

# If not found in common subdirectories, search more broadly within extract_dir
if not images_dir:
    for root, dirs, files in os.walk(extract_dir):
        if any(f.lower().endswith(('.jpg', '.png', '.jpeg', '.webp')) for f in files):
            images_dir = Path(root)
            break

if not images_dir:
    print("⚠️ No se encontró una carpeta de imágenes específica. Se usará el directorio de extracción completo para la búsqueda.")
    images_dir = Path(extract_dir)

print(f"✅ Carpeta de imágenes identificada: {images_dir}")

# ================= 5. LISTAR IMÁGENES =================
imagenes = []
for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.PNG']:
    imagenes.extend(list(images_dir.rglob(ext)))

if not imagenes:
    raise FileNotFoundError(f"❌ No se encontraron imágenes en: {images_dir}")

print(f"📸 Total imágenes encontradas para procesamiento: {len(imagenes)}")
for img_path in imagenes[:5]:
    print(f"  - {img_path.name}")

# ================= 6. CARGAR JSON COCO =================
with open(json_path, 'r') as f:
    coco_data = json.load(f)

print(f"📊 Total de imágenes en JSON (COCO): {len(coco_data.get('images', []))}")
print(f"📊 Total de anotaciones en JSON (COCO): {len(coco_data.get('annotations', []))}")
print("✅ `coco_data`, `imagenes`, `json_path`, `images_dir`, y `extract_dir` inicializados.")

## Consolidate Library Installations and NumPy Downgrade

### Subtask:
Gather all necessary Python library installations (`imgaug`, `opencv-python`, `scikit-image`) and the `numpy` downgrade into a single, clean cell. This centralizes dependency management and ensures `imgaug` compatibility from the start.


**Reasoning**:
The user wants to consolidate all library installations and the `numpy` downgrade into a single code block. This will first uninstall any existing numpy, then install `numpy==1.26.4` for `imgaug` compatibility, and finally install `imgaug`, `opencv-python`, and `scikit-image`.



In [ ]:
print('🔄 Downgrading numpy for imgaug compatibility and installing other libraries...')
# Uninstall existing numpy
!pip uninstall -y numpy
# Install compatible numpy version
!pip install numpy==1.26.4
# Install other necessary libraries
!pip install imgaug opencv-python scikit-image
print('✅ Libraries installed and numpy downgraded.')

**Reasoning**:
The previous attempt to downgrade numpy failed because `imgaug` re-installed a newer version. To ensure `numpy==1.26.4` remains installed, I will first install `imgaug` and other libraries to satisfy their dependencies, and then explicitly reinstall `numpy==1.26.4` using `--no-deps` to prevent its version from being overwritten.



In [ ]:
print('🔄 Ensuring imgaug compatibility and installing other libraries...')

# Uninstall any numpy version first to ensure a clean state
!pip uninstall -y numpy

# Install imgaug and other libraries, letting them install their preferred numpy (if any)
!pip install imgaug opencv-python scikit-image

# Force install numpy==1.26.4, without re-evaluating its dependencies
# This ensures imgaug's numpy dependency (>=1.15) is met, but specifically locks to 1.26.4
!pip install numpy==1.26.4 --no-deps

print('✅ Libraries installed and numpy downgraded to 1.26.4 successfully.')

## Refactor and Generate Augmented Segmentation Dataset

### Subtask:
Refactor the code responsible for generating synthetic images and their COCO annotations. This step will take the initial dataset and apply a series of geometric and photometric transformations using `imgaug` to create an augmented dataset (approximately 100 images). The code will be cleaned for readability and efficiency, ensuring that images and their corresponding updated COCO JSON files (`filtered_coco.json`, `dataset_info.json`) are correctly saved in a new `augmented_dataset` directory.


**Reasoning**:
The user explicitly asked to execute the code in cell `bd8e361b` to generate the augmented dataset. This code leverages the `coco_data`, `imagenes`, `json_path`, `images_dir`, and `extract_dir` variables that were successfully initialized in the consolidated setup cell `212a5494`.



In [ ]:
ia.seed(1)

augmented_coco_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data.get("categories", []),
    "info": coco_data.get("info", {})
}
augmented_dataset_info_list = []

# Keep track of new IDs for images and annotations
existing_image_ids = [img['id'] for img in coco_data.get('images', [])]
new_image_id = max(existing_image_ids) + 1 if existing_image_ids else 0

existing_annotation_ids = [ann['id'] for ann in coco_data.get('annotations', [])]
new_annotation_id = max(existing_annotation_ids) + 1 if existing_annotation_ids else 0

num_augmentations_per_image = 9 # Generate 9 augmented versions for each original image

print(f"\n--- Generando dataset aumentado (aproximadamente {len(original_images_info) * (1 + num_augmentations_per_image)} imágenes) ---")

# Iterar sobre cada imagen original
for original_img_id, original_img_info in original_images_info.items():
    original_file_name = original_img_info.get('file_name')
    original_width = original_img_info.get('width')
    original_height = original_img_info.get('height')

    if not original_file_name or original_width is None or original_height is None:
        print(f"⚠️ Información incompleta para imagen original ID {original_img_id}. Saltando.")
        continue

    # Find the actual path of the original image from the 'imagenes' list
    original_img_path = None
    nombre_limpio = os.path.basename(str(original_file_name))
    if '?' in nombre_limpio:
        nombre_limpio = nombre_limpio.split('?')[0]

    for img_path_candidate in imagenes:
        if img_path_candidate.name == nombre_limpio:
            original_img_path = img_path_candidate
            break

    if original_img_path is None:
        print(f"❌ Imagen original '{nombre_limpio}' (ID: {original_img_id}) no encontrada en `imagenes`. Saltando.")
        continue

    # Load original image
    image_original = cv2.imread(str(original_img_path))
    if image_original is None:
        print(f"❌ No se pudo cargar la imagen: {original_img_path}. Saltando.")
        continue
    image_original = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB) # Convert to RGB for imgaug

    # Process original annotations for imgaug
    segmentations_ia = []
    original_annotations = original_annotations_info.get(original_img_id, [])
    for ann in original_annotations:
        if 'segmentation' in ann and ann['segmentation']:
            if isinstance(ann['segmentation'], list) and len(ann['segmentation']) > 0:
                for poly_coords in ann['segmentation']:
                    points = np.array(poly_coords).reshape(-1, 2)
                    segmentations_ia.append(ia.Polygon(points, label=ann['category_id']))

    # --- Handle the original image itself (always copy if successfully loaded) ---
    # Use original COCO image ID as the basis for the filename
    original_image_saved_name = f"{original_img_id}{original_img_path.suffix}"
    cv2.imwrite(str(augmented_img_dir / original_image_saved_name), cv2.cvtColor(image_original, cv2.COLOR_RGB2BGR))

    # Create a new image info dictionary for the COCO format, ensuring file_name points to the copied file
    original_img_coco_entry = original_img_info.copy() # Make a copy to avoid modifying original_images_info
    original_img_coco_entry['file_name'] = original_image_saved_name # Update filename to match the saved one
    augmented_coco_data["images"].append(original_img_coco_entry)

    # Add original annotations
    for ann in original_annotations:
        augmented_coco_data["annotations"].append(ann)

    augmented_dataset_info_list.append({
        'id': original_img_id, # Use original ID for this entry
        'aligned_image': original_image_saved_name, # This is the key for AlopeciaDataset
        'annotation_count': len(original_annotations)
    })
    print(f"  Añadida imagen original ID {original_img_id} ('{original_file_name}') a dataset aumentado.")

    # Only augment if there are segmentation annotations for the image
    if not segmentations_ia:
        print(f"⚠️ Imagen ID {original_img_id} ('{original_file_name}') no tiene anotaciones de segmentación válidas para aumentación. Solo se añadió la versión original.")
        continue # Skip augmentation for this image if no valid segmentations

    # Apply augmentations
    for aug_idx in range(num_augmentations_per_image):
        seq_det = seq.to_deterministic()
        image_aug = seq_det.augment_image(image_original)

        # Wrap segmentations_ia in PolygonsOnImage for augmentation
        polygons_on_image = ia.PolygonsOnImage(segmentations_ia, shape=image_original.shape)
        polygons_aug_on_image = seq_det.augment_polygons([polygons_on_image])[0] # Augment and unwrap from batch
        polygons_aug = polygons_aug_on_image.polygons

        # Generate new filenames and IDs
        # Ensure augmented IDs are distinct from original IDs and from each other
        current_new_image_id = new_image_id
        new_image_id += 1 # Increment for the next augmented image
        new_image_filename = f"{original_img_id}_aug{aug_idx}{original_img_path.suffix}" # Use original COCO ID as base

        # Save augmented image
        cv2.imwrite(str(augmented_img_dir / new_image_filename), cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))

        # Create new COCO image entry for augmented image
        new_img_coco_info = {
            "id": current_new_image_id,
            "file_name": new_image_filename,
            "width": original_width,
            "height": original_height
        }
        augmented_coco_data["images"].append(new_img_coco_info)

        # Create new COCO annotation entries for augmented polygons
        num_augmented_annotations = 0
        for poly_ia in polygons_aug:
            if poly_ia.is_valid and poly_ia.area > 0 and len(poly_ia.xx) >= 3:
                # Convert augmented polygon back to COCO segmentation format
                coco_segmentation = np.column_stack([poly_ia.xx, poly_ia.yy]).flatten().tolist()
                # Ensure all elements in coco_segmentation are standard Python floats
                coco_segmentation = [float(p) for p in coco_segmentation]

                # Get bounding box from the polygon object
                bbox_obj = poly_ia.to_bounding_box()
                x_min, y_min, x_max, y_max = bbox_obj.x1, bbox_obj.y1, bbox_obj.x2, bbox_obj.y2
                bbox_aug = [float(x_min), float(y_min), float(x_max - x_min), float(y_max - y_min)]

                new_ann_coco_info = {
                    "id": new_annotation_id,
                    "image_id": current_new_image_id,
                    "category_id": int(poly_ia.label), # Category ID is stored in label
                    "segmentation": [coco_segmentation],
                    "area": float(poly_ia.area), # Ensure JSON serializable float
                    "bbox": bbox_aug,
                    "iscrowd": 0
                }
                augmented_coco_data["annotations"].append(new_ann_coco_info)
                new_annotation_id += 1 # Increment for the next annotation
                num_augmented_annotations += 1

        augmented_dataset_info_list.append({
            'id': current_new_image_id,
            'aligned_image': new_image_filename,
            'annotation_count': num_augmented_annotations
        })
        print(f"    Generada aumentada {aug_idx+1}/{num_augmentations_per_image} (ID: {current_new_image_id}) con {num_augmented_annotations} anotaciones.")


# Save the augmented COCO JSON
final_augmented_coco_path = augmented_annot_dir / 'filtered_coco.json'
with open(final_augmented_coco_path, 'w') as f:
    json.dump(augmented_coco_data, f, indent=4)
print(f"\n✅ 'filtered_coco.json' para el dataset aumentado creado en: {final_augmented_coco_path}")

# Save the augmented dataset_info.json
final_dataset_info_path = augmented_annot_dir / 'dataset_info.json'
with open(final_dataset_info_path, 'w') as f:
    json.dump(augmented_dataset_info_list, f, indent=4)
print(f"✅ 'dataset_info.json' para el dataset aumentado creado en: {final_dataset_info_path}")

print(f"\n🎉 Generación del dataset aumentado completada.")
print(f"Total de imágenes en el dataset aumentado: {len(augmented_coco_data['images'])}")
print(f"Total de anotaciones en el dataset aumentado: {len(augmented_coco_data['annotations'])}")

## Refactor and Visualize Model Predictions

### Subtask:
Refactor the code for visualizing model predictions. This step will load the trained segmentation model and allow for testing with a new image (e.g., from Google Drive). The image preprocessing, prediction, and visualization of the original image, probability map, and binary mask will be presented in a clean and concise manner.


**Reasoning**:
I will create a single code block that includes all the necessary steps to refactor and visualize model predictions. This includes importing libraries, defining the `SimpleUNet` model, loading the trained weights, preparing a test image for inference, running the prediction, and displaying the results with original image, probability map, and binary mask.



In [ ]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

# 1. & 2. Definir la clase SimpleUNet (copiada del código de entrenamiento)
class SimpleUNet(nn.Module):
    def __init__(self):
        super(SimpleUNet, self).__init__()

        # Encoder
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )

        # Decoder
        self.up1 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(192, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU()
        )
        self.up2 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(67, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU()
        )

        # Salida (logits, sin sigmoid)
        self.output = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        b = self.bottleneck(e2)

        d1 = self.up1(b)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)

        d2 = self.up2(d1)
        d2 = torch.cat([d2, x[:, :, :d2.shape[2], :d2.shape[3]]], dim=1)
        d2 = self.dec2(d2)

        out = self.output(d2)
        return out # <-- Dejamos los logits crudos

# 3. & 4. & 5. & 6. Cargar el modelo entrenado
model_path = '/content/alopecia_dataset/alopecia_segmentation_model.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SimpleUNet().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval() # Poner el modelo en modo evaluación

print(f"✅ Modelo cargado exitosamente desde: {model_path} y puesto en modo evaluación.")

# 7. & 8. & 9. & 10. & 11. Cargar y preprocesar una nueva imagen
# Asegurarse de que `imagenes` y `images_dir` estén disponibles desde las celdas anteriores
# Para esta demostración, usaremos la primera imagen encontrada en el paso de extracción.
if 'imagenes' not in locals() or not imagenes:
    print("⚠️ `imagenes` no está definido o está vacío. Re-listando imágenes...")
    # Re-define paths if not globally available, assuming previous cell execution state
    json_path = '/content/temp_extract/test-coco/result_coco.json'
    images_dir = '/content/temp_extract/test-coco/images'

    imagenes = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.PNG']:
        imagenes.extend(list(Path(images_dir).rglob(ext)))
    if not imagenes:
        raise FileNotFoundError("No se encontraron imágenes en el directorio extraído para realizar la predicción.")

new_image_path = str(imagenes[0]) # Usar la ruta de la primera imagen encontrada
img_size = 256 # El tamaño que usamos para entrenar

# Cargar la imagen
new_img_bgr = cv2.imread(new_image_path)

if new_img_bgr is None:
    raise FileNotFoundError(f"❌ No se pudo cargar la imagen: {new_image_path}")

# Redimensionar y normalizar
new_img_resized = cv2.resize(new_img_bgr, (img_size, img_size))
new_img_rgb = cv2.cvtColor(new_img_resized, cv2.COLOR_BGR2RGB)

# Convertir a tensor PyTorch y preprocesar
input_tensor = torch.tensor(new_img_rgb).permute(2, 0, 1).float() / 255.0
input_tensor = input_tensor.unsqueeze(0).to(device) # Añadir dimensión de batch y mover a dispositivo

print(f"✅ Imagen '{os.path.basename(new_image_path)}' cargada y preprocesada para predicción.")

# 12. & 13. Realizar la predicción y visualizar
with torch.no_grad():
    # Obtener predicción cruda (logits)
    logits = model(input_tensor)

    # Convertir a probabilidad (0 a 1) usando Sigmoid
    probs = torch.sigmoid(logits)

# Procesar resultados para visualizar
# Quitamos las dimensiones extra (batch, channels) para tener solo (256, 256)
probs_map = probs.squeeze().cpu().numpy()
mask_binary = (probs_map > 0.3).astype(np.uint8) # Umbral de 0.3 para la máscara binaria

# Visualizar
plt.figure(figsize=(15, 5))

# Imagen Original
plt.subplot(1, 3, 1)
plt.imshow(new_img_rgb)
plt.title('Imagen Original')
plt.axis('off')

# Mapa de Calor (Probabilidad)
plt.subplot(1, 3, 2)
im = plt.imshow(probs_map, cmap='jet', vmin=0, vmax=1)
plt.title('Mapa de Calor (Confianza)')
plt.axis('off')
plt.colorbar(im, fraction=0.046, pad=0.04)

# Máscara Final
plt.subplot(1, 3, 3)
plt.imshow(mask_binary, cmap='gray')
plt.title('Predicción Final (> 0.3)')
plt.axis('off')

plt.tight_layout()
plt.show()

print("✅ Visualización de la predicción completada.")

**Reasoning**:
The previous execution ended with a KeyboardInterrupt, which typically indicates a manual interruption rather than a code error. The code for visualizing model predictions already appears correct and complete based on the instructions. I will re-output the same code block to ensure it is executed without interruption.



In [ ]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

# 1. & 2. Definir la clase SimpleUNet (copiada del código de entrenamiento)
class SimpleUNet(nn.Module):
    def __init__(self):
        super(SimpleUNet, self).__init__()

        # Encoder
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )

        # Decoder
        self.up1 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(192, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU()
        )
        self.up2 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(67, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU()
        )

        # Salida (logits, sin sigmoid)
        self.output = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(e1)
        b = self.bottleneck(e2)

        d1 = self.up1(b)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)

        d2 = self.up2(d1)
        d2 = torch.cat([d2, x[:, :, :d2.shape[2], :d2.shape[3]]], dim=1)
        d2 = self.dec2(d2)

        out = self.output(d2)
        return out # <-- Dejamos los logits crudos

# 3. & 4. & 5. & 6. Cargar el modelo entrenado
model_path = '/content/alopecia_dataset/alopecia_segmentation_model.pth'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SimpleUNet().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval() # Poner el modelo en modo evaluación

print(f"✅ Modelo cargado exitosamente desde: {model_path} y puesto en modo evaluación.")

# 7. & 8. & 9. & 10. & 11. Cargar y preprocesar una nueva imagen
# Asegurarse de que `imagenes` y `images_dir` estén disponibles desde las celdas anteriores
# Para esta demostración, usaremos la primera imagen encontrada en el paso de extracción.
if 'imagenes' not in locals() or not imagenes:
    print("⚠️ `imagenes` no está definido o está vacío. Re-listando imágenes...")
    # Re-define paths if not globally available, assuming previous cell execution state
    json_path = '/content/temp_extract/test-coco/result_coco.json'
    images_dir = '/content/temp_extract/test-coco/images'

    imagenes = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.webp', '*.JPG', '*.PNG']:
        imagenes.extend(list(Path(images_dir).rglob(ext)))
    if not imagenes:
        raise FileNotFoundError("No se encontraron imágenes en el directorio extraído para realizar la predicción.")

new_image_path = str(imagenes[0]) # Usar la ruta de la primera imagen encontrada
img_size = 256 # El tamaño que usamos para entrenar

# Cargar la imagen
new_img_bgr = cv2.imread(new_image_path)

if new_img_bgr is None:
    raise FileNotFoundError(f"❌ No se pudo cargar la imagen: {new_image_path}")

# Redimensionar y normalizar
new_img_resized = cv2.resize(new_img_bgr, (img_size, img_size))
new_img_rgb = cv2.cvtColor(new_img_resized, cv2.COLOR_BGR2RGB)

# Convertir a tensor PyTorch y preprocesar
input_tensor = torch.tensor(new_img_rgb).permute(2, 0, 1).float() / 255.0
input_tensor = input_tensor.unsqueeze(0).to(device) # Añadir dimensión de batch y mover a dispositivo

print(f"✅ Imagen '{os.path.basename(new_image_path)}' cargada y preprocesada para predicción.")

# 12. & 13. Realizar la predicción y visualizar
with torch.no_grad():
    # Obtener predicción cruda (logits)
    logits = model(input_tensor)

    # Convertir a probabilidad (0 a 1) usando Sigmoid
    probs = torch.sigmoid(logits)

# Procesar resultados para visualizar
# Quitamos las dimensiones extra (batch, channels) para tener solo (256, 256)
probs_map = probs.squeeze().cpu().numpy()
mask_binary = (probs_map > 0.3).astype(np.uint8) # Umbral de 0.3 para la máscara binaria

# Visualizar
plt.figure(figsize=(15, 5))

# Imagen Original
plt.subplot(1, 3, 1)
plt.imshow(new_img_rgb)
plt.title('Imagen Original')
plt.axis('off')

# Mapa de Calor (Probabilidad)
plt.subplot(1, 3, 2)
im = plt.imshow(probs_map, cmap='jet', vmin=0, vmax=1)
plt.title('Mapa de Calor (Confianza)')
plt.axis('off')
plt.colorbar(im, fraction=0.046, pad=0.04)

# Máscara Final
plt.subplot(1, 3, 3)
plt.imshow(mask_binary, cmap='gray')
plt.title('Predicción Final (> 0.3)')
plt.axis('off')

plt.tight_layout()
plt.show()

print("✅ Visualización de la predicción completada.")


## Refactor and Train Segmentation Model with Augmented Data

### Subtask:
Consolidate and refactor the AlopeciaDataset class, the SimpleUNet model definition, and the training loop. This step will use the newly generated augmented dataset for training. The code will be optimized for clarity and robustness, ensuring that the model trains for 30 epochs, saves checkpoints, and outputs the final trained model.


**Reasoning**:
The subtask requires consolidating and refactoring the `AlopeciaDataset` class, `SimpleUNet` model, and the training loop into a single code block, pointing to the augmented dataset, and ensuring all specified training and visualization steps are included.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import json

# ----------------------------
# 1. DATASET PARA ENTRENAMIENTO
# ----------------------------
class AlopeciaDataset(Dataset):
    def __init__(self, aligned_dir, img_size=256):
        self.img_dir = os.path.join(aligned_dir, 'images')
        self.annot_dir = os.path.join(aligned_dir, 'annotations')
        self.img_size = img_size

        # Cargar información del dataset
        info_path = os.path.join(aligned_dir, 'annotations', 'dataset_info.json')
        with open(info_path, 'r') as f:
            self.dataset_info = json.load(f)

        # También cargar COCO filtrado
        coco_path = os.path.join(aligned_dir, 'annotations', 'filtered_coco.json')
        with open(coco_path, 'r') as f:
            self.coco_data = json.load(f)

        print(f"Dataset cargado: {len(self.dataset_info)} imágenes, {sum(item['annotation_count'] for item in self.dataset_info)} anotaciones")

    def __len__(self):
        return len(self.dataset_info)

    def __getitem__(self, idx):
        item = self.dataset_info[idx]
        img_path = os.path.join(self.img_dir, item['aligned_image'])

        # Cargar imagen
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"No se pudo cargar la imagen: {img_path}. Asegúrate de que el archivo existe.")

        img = cv2.resize(img, (self.img_size, self.img_size))
        img = img / 255.0  # Normalizar [0, 1]
        img = torch.tensor(img).permute(2, 0, 1).float()

        # Crear máscara
        mask = self._create_mask_from_coco(item['id'], self.img_size)
        mask = torch.tensor(mask).unsqueeze(0).float()

        return img, mask

    def _create_mask_from_coco(self, image_id, size):
        mask = np.zeros((size, size), dtype=np.uint8)

        # Get image info
        image_info = None
        for img_info in self.coco_data['images']:
            if img_info['id'] == image_id:
                image_info = img_info
                break
        if image_info is None:
            raise ValueError(f"Image info not found for image_id: {image_id}")

        img_width = image_info['width']
        img_height = image_info['height']

        # Buscar anotaciones
        for ann in self.coco_data['annotations']:
            if ann['image_id'] == image_id:
                seg = ann['segmentation']
                if isinstance(seg, list):
                    for polygon in seg:
                        pts = np.array(polygon).reshape(-1, 2)
                        # Scale points to the target size, considering aspect ratio if necessary
                        # For simplicity, assuming direct scaling of points for now, adjust as needed
                        pts = (pts * (size / max(img_width, img_height))).astype(np.int32)
                        # Ensure points are within bounds
                        pts[:, 0] = np.clip(pts[:, 0], 0, size - 1)
                        pts[:, 1] = np.clip(pts[:, 1], 0, size - 1)

                        # Only fill if there are enough points to form a polygon
                        if len(pts) >= 3:
                            cv2.fillPoly(mask, [pts], 1)

        return mask

# ----------------------------
# 2. MODELO MEJORADO (Sin Sigmoid al final)
# ----------------------------
class SimpleUNet(nn.Module):
    def __init__(self):
        super(SimpleUNet, self).__init__()

        # Encoder
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )

        # Decoder
        # The `enc1` output size is (N, 64, H/2, W/2)
        # The `up1` output size is (N, 128, H/2, W/2) given input (N, 256, H/4, W/4)
        # Concatenation should be (128 + 64 = 192)
        self.up1 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv2d(192, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU()
        )

        # The `x` (original input) size is (N, 3, H, W)
        # The `up2` output size is (N, 64, H, W) given input (N, 128, H/2, W/2)
        # Concatenation should be (64 + 3 = 67)
        self.up2 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv2d(67, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1), nn.ReLU()
        )

        # Salida (logits, sin sigmoid)
        self.output = nn.Conv2d(64, 1, 1)

    def forward(self, x):
        e1 = self.enc1(x) # Output size: N, 64, H/2, W/2
        e2 = self.enc2(e1) # Output size: N, 128, H/4, W/4
        b = self.bottleneck(e2) # Output size: N, 256, H/4, W/4

        d1 = self.up1(b) # Output size: N, 128, H/2, W/2
        d1 = torch.cat([d1, e1], dim=1) # Concatenate with e1, resulting in N, 192, H/2, W/2
        d1 = self.dec1(d1) # Output size: N, 128, H/2, W/2

        d2 = self.up2(d1) # Output size: N, 64, H, W
        d2 = torch.cat([d2, x[:, :, :d2.shape[2], :d2.shape[3]]], dim=1) # Concatenate with original input, resulting in N, 67, H, W
        d2 = self.dec2(d2) # Output size: N, 64, H, W

        out = self.output(d2) # Output size: N, 1, H, W (logits)
        return out # <-- Dejamos los logits crudos

# ----------------------------
# 3. ENTRENAMIENTO (Con pesos para clases desbalanceadas)
# ----------------------------
def train_model():
    # a. Set the aligned_dir variable to point to the augmented dataset path
    aligned_dir = '/content/alopecia_dataset/augmented_dataset'
    # b. Set the device to 'cuda' if available, otherwise 'cpu'.
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Usando dispositivo: {device}")

    # c. Instantiate the AlopeciaDataset with the aligned_dir and an img_size of 256.
    dataset = AlopeciaDataset(aligned_dir, img_size=256)
    # d. Create a DataLoader for the dataset with a batch_size of 2 and shuffle=True.
    dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

    # e. Instantiate the SimpleUNet model and move it to the device.
    model = SimpleUNet().to(device)

    # f. Define the pos_weight for BCEWithLogitsLoss as a torch.tensor([10.0]) and move it to the device.
    pos_weight = torch.tensor([10.0]).to(device)
    # g. Instantiate nn.BCEWithLogitsLoss with the defined pos_weight.
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    # h. Instantiate the Adam optimizer for the model parameters with a learning rate of 0.001.
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # i. Set num_epochs to 30.
    num_epochs = 30
    train_losses = []

    print("\n=== COMIENZO DEL ENTRENAMIENTO CON DATA AUMENTADA ===")

    # j. Implement the training loop
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0

        for batch_idx, (images, masks) in enumerate(dataloader):
            # iii. Moving images and masks to the device.
            images = images.to(device)
            masks = masks.to(device)

            # iv. Performing a forward pass (model(images)).
            outputs = model(images)
            # v. Calculating the loss (criterion(outputs, masks)).
            loss = criterion(outputs, masks)

            # vi. Zeroing gradients (optimizer.zero_grad()).
            optimizer.zero_grad()
            # vii. Performing backpropagation (loss.backward()).
            loss.backward()
            # viii. Updating model weights (optimizer.step()).
            optimizer.step()

            # ix. Accumulating and printing batch loss.
            epoch_loss += loss.item()

            if batch_idx % 20 == 0: # Print less frequently for augmented dataset
                 print(f"  Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(dataloader)}: Loss = {loss.item():.4f}")

        # x. Calculating and printing the average epoch loss.
        avg_loss = epoch_loss / len(dataloader)
        train_losses.append(avg_loss)

        print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}")

        # xi. Saving a model checkpoint every 5 epochs
        if (epoch + 1) % 5 == 0:
            checkpoint_path = f'/content/alopecia_dataset/model_checkpoint_epoch_{epoch+1}.pth'
            torch.save(model.state_dict(), checkpoint_path)
            print(f"  Checkpoint guardado: {checkpoint_path}")

    # k. Save the final trained model
    final_model_path = '/content/alopecia_dataset/alopecia_segmentation_model.pth'
    torch.save(model.state_dict(), final_model_path)
    print(f"\n✅ Modelo final guardado: {final_model_path}")

    # l. Return the trained model and the list of epoch losses.
    return model, train_losses

# ----------------------------
# 4. VISUALIZACIÓN MEJORADA (Mapas de calor)
# ----------------------------
def visualize_predictions(model, dataset, num_samples=3):
    model.eval()
    device = next(model.parameters()).device

    fig, axes = plt.subplots(num_samples, 4, figsize=(16, 4*num_samples))
    if num_samples == 1: # Adjust axes indexing if only one sample
        axes = axes.reshape(1, -1)

    for i in range(num_samples):
        img, true_mask = dataset[i]
        img_tensor = img.unsqueeze(0).to(device)

        with torch.no_grad():
            logits = model(img_tensor)
            probs = torch.sigmoid(logits) # Convertimos logits a prob (0-1)
            pred_mask = (probs > 0.3).float() # Umbral bajado a 0.3

        img_np = img.permute(1, 2, 0).cpu().numpy()
        true_mask_np = true_mask[0].cpu().numpy()
        probs_np = probs[0][0].cpu().numpy()
        pred_mask_np = pred_mask[0][0].cpu().numpy()

        # Imagen
        axes[i, 0].imshow(img_np)
        axes[i, 0].set_title(f"Imagen {i+1}")
        axes[i, 0].axis('off')

        # Real
        axes[i, 1].imshow(true_mask_np, cmap='gray')
        axes[i, 1].set_title("Real")
        axes[i, 1].axis('off')

        # Calor (Probabilidades)
        im = axes[i, 2].imshow(probs_np, cmap='jet', vmin=0, vmax=1)
        axes[i, 2].set_title("Confianza (Mapa Calor)")
        axes[i, 2].axis('off')
        fig.colorbar(im, ax=axes[i, 2], fraction=0.046, pad=0.04)

        # Predicción binaria
        axes[i, 3].imshow(pred_mask_np, cmap='gray')
        axes[i, 3].set_title("Predicción (>0.3)")
        axes[i, 3].axis('off')

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    print("🚀 INICIANDO ENTRENAMIENTO DEL MODELO DE SEGMENTACIÓN")
    try:
        # a. Call train_model() to start the training process.
        trained_model, losses = train_model()

        # b. Plot the training loss curve using matplotlib.pyplot.
        plt.figure(figsize=(10, 4))
        plt.plot(losses)
        plt.xlabel('Época')
        plt.ylabel('Pérdida')
        plt.title('Curva de Aprendizaje')
        plt.grid(True)
        plt.show()

        # c. Re-instantiate AlopeciaDataset with the augmented dataset path for visualization purposes.
        aligned_dir = '/content/alopecia_dataset/augmented_dataset'
        dataset = AlopeciaDataset(aligned_dir, img_size=256)

        print("\n🎯 VISUALIZANDO PREDICCIONES DEL MODELO")
        # d. Call visualize_predictions() with the trained model and the dataset to display sample predictions.
        visualize_predictions(trained_model, dataset, num_samples=min(3, len(dataset)))

        print("\n✅ ¡ENTRENAMIENTO COMPLETADO!")
    except Exception as e:
        print(f"❌ Error durante la ejecución: {e}")
        import traceback
        traceback.print_exc()


## Final Task

### Subtask:
Summarize the entire process, confirming the successful generation of synthetic data, the training of the segmentation model with this augmented dataset, and any observed improvements or next steps.


## Summary:

### Data Analysis Key Findings

*   **Initial Data Setup:** Google Drive was successfully mounted, and the COCO dataset (`test-coco-upload.zip`) containing 12 images and 19 annotations was extracted and loaded for initial processing.
*   **Dependency Management:** All required libraries (`imgaug`, `opencv-python`, `scikit-image`) were successfully installed, and a critical `numpy` downgrade to version `1.26.4` was performed to ensure compatibility with `imgaug`, resolving previous `AttributeError` issues.
*   **Synthetic Data Generation:** The original dataset was successfully augmented, expanding the dataset from 12 original images to **110 images** with a total of **180 annotations**. This process involved applying various geometric and photometric transformations using `imgaug`, with several technical issues related to JSON serialization and polygon handling successfully resolved.
*   **Segmentation Model Training:** A `SimpleUNet` model was successfully trained for **30 epochs** using the newly generated augmented dataset. Training checkpoints were saved every 5 epochs, and the final trained model was stored as `/content/alopecia_dataset/alopecia_segmentation_model.pth`. The training loss curve indicated learning progression.
*   **Model Prediction Visualization:** The trained segmentation model was successfully loaded, and its predictions for sample images were visualized. This included displaying the original image, a probability map, and a binary mask, confirming the model's ability to generate segmentations.

### Insights or Next Steps

*   **Leverage Augmented Data for Robustness:** The significant increase in dataset size (from 12 to 110 images) through augmentation provides a more robust foundation for training, likely leading to a more generalized segmentation model for alopecia detection.
*   **Further Model Evaluation and Optimization:** The current setup provides a functional model. The next steps should involve a detailed quantitative evaluation (e.g., using metrics like IoU, Dice Score) on a dedicated validation set, followed by hyperparameter tuning and potentially exploring more advanced UNet architectures or transfer learning to further improve performance.
